## [Semeval Task 6.1 - CLARITY - Unmasking Political Question Evasions](https://semeval.github.io/SemEval2026/tasks.html)

### Team members:
 * Oprea Stefan
 * Toma Oprea Lacrimioara

### State of the art (AI generated)

#### 1. Description of the Problem

##### 1.1 Context and Motivation

In democratic systems, political accountability relies on the ability of journalists and the public to elicit clear information from elected officials through Question-Answering (Q&A) sessions. However, in high-stakes environments like presidential debates or press conferences, politicians often employ **equivocation**—a strategic communication style characterized by being intentionally non-straightforward or ambiguous.

The **CLARITY** task (SemEval-2026 Task 6) addresses the computational gap in detecting these dodges. While evasion has been studied for decades in political science, it remains a "frontier" problem for Natural Language Processing (NLP) due to the subtle, high-level semantic reasoning required to distinguish between a "lengthy but informative" reply and a "lengthy but evasive" one.

##### 1.2 Task Definition

The core problem is defined as a hierarchical classification of response quality given a specific question context.

* **Clarity-Level Classification (Level 1):** A three-way classification task where a response must be labeled as:
* **Clear Reply:** The response directly addresses the question constraints.
* **Ambiguous:** The response partially addresses the question or uses vague language.
* **Clear Non-Reply:** The response is a total evasion or a refusal to answer.


* **Evasion-Level Classification (Level 2):** A fine-grained classification into 9 distinct evasion techniques. Based on the **Thomas et al. (2024)** taxonomy, these include:
* *Attacking the questioner*
* *Answering a different question*
* *Questioning the question’s premise*
* *Pleading ignorance or confidentiality*
* *Declining to answer* (and others).



---

#### 2. Review of the State of the Art (SOTA)

##### 2.1 Theoretical Foundations

Research in this field originated with **Bavelas et al. (1990)** and the **Equivocation Theory**, later refined by **Bull and Mayer (1993)**, who identified 30 manual tactics used by politicians to elude questions. For decades, analysis was limited to manual coding by political scientists, which lacked the scalability needed for modern digital media.

##### 2.2 Current Computational Approaches (2024–2025)

State-of-the-art approaches have recently shifted from keyword-based irrelevance detection to deep contextual reasoning:

* **Large Language Models (LLMs):** Current benchmarks utilize models like **GPT-4o, Llama 3.1, and Mistral 7B**. Recent findings (January 2026) indicate that **Chain-of-Thought (CoT)** prompting is essential for this task; models that explicitly "reason" through the question's core request before classifying the answer show significant accuracy gains (up to 74% in topic alignment).
* **Cross-Encoder Architectures:** Unlike Bi-Encoders that process  and  separately, SOTA systems for CLARITY often use **DeBERTa-v3** in a Cross-Encoder setup. This allows the model to capture the fine-grained interaction between the  and  tokens, identifying "pivots" where the politician shifts from the topic.
* **Natural Language Inference (NLI) Framing:** Modern systems often treat evasion as an "entailment" problem. If the response does not logically entail a direct answer to the question's presuppositions, it is flagged. Research by **Thomas et al. (EMNLP 2024)** demonstrated that hierarchical models (predicting Level 1 and Level 2 simultaneously) outperform independent classifiers.
* **Quality Metrics beyond F1:** The current SOTA also investigates "Answer Quality" through **self-supervised learning**. Models are trained to rank "observed" answers against "distractor" answers; if an observed answer is easily confused with a random one, it is quantitatively marked as having low semantic engagement (evasive).

---

### Approach

We will use several classifiers, trained by us as well as pretrained, and use a voting system:
 * if two thirds of classifiers agree -> return majority label
 * else -> return 'Ambivalent' label

```mermaid
sequenceDiagram
    user->>runner: submit dataset
    runner->>runner: format dataset
    runner->>classifier1: submit batch task
    classifier1->>runner: return prediction file
    runner->>classifier2: submit batch task
    classifier2->>runner: return prediction file
    runner->>runner: tie breaker
    runner->>user: return prediction file
```

#### API contract

```yaml
request:
  lines:
    type: array
    items:
      type: object
      items:
        interview_question: text
        interview_answer: text
response:
  lines:
    type: array
    items:
      type: object
      items:
        clarity_label: text
```

#### Datasets

 * [training dataset](https://huggingface.co/datasets/ailsntua/QEvasion)
 * [evaluation dataset](https://github.com/konstantinosftw/CLARITY-SemEval-2026/blob/main/dataset/clarity_task_evaluation_dataset.csv)

 #### First model

 Fine-tuning pretrained BERT model (BertForSequenceClassification)

  * https://huggingface.co/docs/transformers/en/tasks/sequence_classification
  * https://huggingface.co/transformers/v3.0.2/model_doc/bert.html#bertforsequenceclassification



In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ailsntua/QEvasion")


DatasetDict({
    train: Dataset({
        features: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label'],
        num_rows: 3448
    })
    test: Dataset({
        features: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label'],
        num_rows: 308
    })
})

In [ ]:
[item['clarity_label'] for item in ds['test'] if item['multiple_questions']]


['Clear Reply', 'Clear Reply']

In [ ]:
!pip install evaluate

from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("ailsntua/QEvasion") #.with_format("torch")
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

from transformers import AutoModelForSequenceClassification

id2label = {0: "Ambivalent", 1: "Clear Reply", 2: "Clear Non-Reply"}
label2id = {"Ambivalent": 0, "Clear Reply": 1, "Clear Non-Reply": 2}

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=3,
                                                           id2label=id2label,
                                                           label2id=label2id,

                                                           )

import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # convert the logits to their predicted class
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="semy_classifier",
    eval_strategy="epoch",
    push_to_hub=False,
    remove_unused_columns=True
)


def convert_dataset(ds):
  return [
    {
        "text": item["interview_question"] + " " + item["interview_answer"],
        "label": label2id[item["clarity_label"]],
    }
    for item in ds]

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=convert_dataset(dataset["train"]),
    eval_dataset=convert_dataset(dataset["test"]),
    compute_metrics=compute_metrics,
)
trainer.train()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: You have to specify either input_ids or inputs_embeds

In [ ]:
imdb = load_dataset("imdb")
imdb['test'][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [1]:
!pip install autogluon

from autogluon.tabular import TabularPredictor
import pandas as pd

from datasets import load_dataset

# 1. Load the dataset from Hugging Face
dataset = load_dataset("ailsntua/QEvasion")

# 2. Convert to a single CSV (if multiple splits exist, e.g., 'train', 'test')
# The QEvasion dataset typically has a 'train' split
dataset['train'].to_csv("train.csv")
dataset['test'].to_csv("test.csv")

print("Downloads complete")

label = 'clarity_label'

train_data = pd.read_csv('train.csv')

# List the columns you want to ignore
excluded_features = [
        'title',
        'date',
        'president',
        'url',
        'question_order',
        'gpt3.5_summary',
        'gpt3.5_prediction',
        # 'question',
        'annotator_id',
        'annotator1',
        'annotator2',
        'annotator3',
        'inaudible',
        'multiple_questions',
        'affirmative_questions',
        'index',
        'evasion_label',
]

train_data = train_data.drop(columns=excluded_features)

# 2. Train (AutoGluon does the heavy lifting)
predictor = TabularPredictor(label=label).fit(
    train_data,
    # verbosity = 4, # default is 2, max is 4
    time_limit = 110 * 60, # in seconds
    presets='high_v150' # medium_quality, high_quality, best_quality, extreme_quality
)

# 3. Predict
predictions = predictor.predict('evaluation.csv')
with open("prediction", "w") as f:
  f.write("\n".join(predictions))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter co

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.90M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/259k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/308 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Downloads complete


No path specified. Models will be saved in: "AutogluonModels/ag-20260202_094114"
Preset alias specified: 'high_v150' maps to 'high_quality_v150'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cu126
CUDA Version:       12.6
GPU Memory:         GPU 0: 14.74/14.74 GB
Total GPU Memory:   Free: 14.74 GB, Allocated: 0.00 GB, Total: 14.74 GB
GPU Count:          1
Memory Avail:       9.19 GB / 12.67 GB (72.5%)
Disk Space Avail:   73.01 GB / 112.64 GB (64.8%)
Presets specified: ['high_v150']
Using hyperparameters preset: hyperparameters='zeroshot_2025_12_18_cpu'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs

## Ran Autogluon with default preset medium and time limit 1 minute

Cell output:

```
 Requirement already satisfied: autogluon in /usr/local/lib/python3.12/dist-packages (1.5.0)
Requirement already satisfied: autogluon.core==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: autogluon.features==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon) (1.5.0)
Requirement already satisfied: autogluon.tabular==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: autogluon.multimodal==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon) (1.5.0)
Requirement already satisfied: autogluon.timeseries==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: numpy<2.4.0,>=1.25.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.0.2)
Requirement already satisfied: scipy<1.17,>=1.5.4 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.16.3)
Requirement already satisfied: scikit-learn<1.8.0,>=1.4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.6.1)
Requirement already satisfied: networkx<4,>=3.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.6.1)
Requirement already satisfied: pandas<2.4.0,>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.2.2)
Requirement already satisfied: tqdm<5,>=4.38 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (4.67.1)
Requirement already satisfied: requests in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.32.4)
Requirement already satisfied: matplotlib<3.11,>=3.7.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.10.0)
Requirement already satisfied: boto3<2,>=1.10 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.42.39)
Requirement already satisfied: autogluon.common==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: ray<2.53,>=2.43.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.52.1)
Requirement already satisfied: pyarrow>=15.0.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (18.1.0)
Requirement already satisfied: hyperopt<0.2.8,>=0.2.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (0.2.7)
Requirement already satisfied: stevedore<5.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (5.4.1)
Requirement already satisfied: Pillow<12,>=10.0.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (11.3.0)
Requirement already satisfied: torch<2.10,>=2.6 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.9.0+cu126)
Requirement already satisfied: lightning<2.6,>=2.5.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.5.6)
Requirement already satisfied: transformers<4.58,>=4.51.0 in /usr/local/lib/python3.12/dist-packages (from transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (4.57.6)
Requirement already satisfied: accelerate<2.0,>=0.34.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.12.0)
Requirement already satisfied: fsspec<=2025.3 in /usr/local/lib/python3.12/dist-packages (from fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (2025.3.0)
Requirement already satisfied: jsonschema<4.24,>=4.18 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (4.23.0)
Requirement already satisfied: seqeval<1.3.0,>=1.2.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.2.2)
Requirement already satisfied: evaluate<0.5.0,>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.4.6)
Requirement already satisfied: timm<1.0.7,>=0.9.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.0.3)
Requirement already satisfied: torchvision<0.25.0,>=0.21.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.24.0+cu126)
Requirement already satisfied: scikit-image<0.26.0,>=0.19.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.25.2)
Requirement already satisfied: text-unidecode<1.4,>=1.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.3)
Requirement already satisfied: torchmetrics<1.8,>=1.2.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.7.4)
Requirement already satisfied: omegaconf<2.4.0,>=2.1.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.3.0)
Requirement already satisfied: pytorch-metric-learning<2.9,>=1.3.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.8.1)
Requirement already satisfied: nlpaug<1.2.0,>=1.1.10 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.1.11)
Requirement already satisfied: nltk<3.10,>=3.4.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (3.9.1)
Requirement already satisfied: openmim<0.4.0,>=0.3.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.3.9)
Requirement already satisfied: defusedxml<0.7.2,>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: jinja2<3.2,>=3.0.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (3.1.6)
Requirement already satisfied: tensorboard<3,>=2.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.19.0)
Requirement already satisfied: pytesseract<0.4,>=0.3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.3.13)
Requirement already satisfied: nvidia-ml-py3<8.0,>=7.352.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (7.352.0)
Requirement already satisfied: pdf2image<1.19,>=1.17.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.17.0)
Requirement already satisfied: einx in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.3.0)
Requirement already satisfied: lightgbm<4.7,>=4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (4.6.0)
Requirement already satisfied: huggingface_hub<1.0 in /usr/local/lib/python3.12/dist-packages (from huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (0.36.0)
Requirement already satisfied: xgboost<3.2,>=2.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (3.1.3)
Requirement already satisfied: catboost<1.3,>=1.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (1.2.8)
Requirement already satisfied: fastai<2.9,>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (2.8.6)
Requirement already satisfied: loguru in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.7.3)
Requirement already satisfied: einops<0.9,>=0.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.8.2)
Requirement already satisfied: spacy<3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (3.8.11)
Requirement already satisfied: joblib<1.7,>=1.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.5.3)
Requirement already satisfied: gluonts<0.17,>=0.15.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.16.2)
Requirement already satisfied: statsforecast<2.0.2,>=1.7.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.0.1)
Requirement already satisfied: mlforecast<0.15.0,>=0.14.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.14.0)
Requirement already satisfied: utilsforecast<0.2.12,>=0.2.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.2.11)
Requirement already satisfied: coreforecast<0.0.17,>=0.0.12 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.0.16)
Requirement already satisfied: fugue>=0.9.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.9.6)
Requirement already satisfied: orjson~=3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (3.11.5)
Requirement already satisfied: chronos-forecasting<2.4,>=2.2.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.2.2)
Requirement already satisfied: peft<0.18,>=0.13.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.17.1)
Requirement already satisfied: psutil<7.2.0,>=5.7.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.common==1.5.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (5.9.5)
Requirement already satisfied: pyyaml>=5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.common==1.5.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (6.0.3)
Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.12/dist-packages (from accelerate<2.0,>=0.34.0->autogluon.multimodal==1.5.0->autogluon) (25.0)
Requirement already satisfied: safetensors>=0.4.3 in /usr/local/lib/python3.12/dist-packages (from accelerate<2.0,>=0.34.0->autogluon.multimodal==1.5.0->autogluon) (0.7.0)
Requirement already satisfied: botocore<1.43.0,>=1.42.39 in /usr/local/lib/python3.12/dist-packages (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.42.39)
Requirement already satisfied: jmespath<2.0.0,>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.1.0)
Requirement already satisfied: s3transfer<0.17.0,>=0.16.0 in /usr/local/lib/python3.12/dist-packages (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (0.16.0)
Requirement already satisfied: graphviz in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (0.21)
Requirement already satisfied: plotly in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (5.24.1)
Requirement already satisfied: six in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (1.17.0)
Requirement already satisfied: datasets>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (4.0.0)
Requirement already satisfied: dill in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (0.3.8)
Requirement already satisfied: xxhash in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (3.6.0)
Requirement already satisfied: multiprocess in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (0.70.16)
Requirement already satisfied: pip in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (24.1.2)
Requirement already satisfied: fastdownload<2,>=0.0.5 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.7)
Requirement already satisfied: fastcore>=1.8.0 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.12.7)
Requirement already satisfied: fasttransform>=0.0.2 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.2)
Requirement already satisfied: fastprogress>=0.2.4 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.1.3)
Requirement already satisfied: plum-dispatch in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: cloudpickle in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.1.2)
Requirement already satisfied: aiohttp!=4.0.0a0,!=4.0.0a1 in /usr/local/lib/python3.12/dist-packages (from fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (3.13.3)
Requirement already satisfied: triad>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from fugue>=0.9.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.0.0)
Requirement already satisfied: adagio>=0.2.6 in /usr/local/lib/python3.12/dist-packages (from fugue>=0.9.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.2.6)
Requirement already satisfied: pydantic<3,>=1.7 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.12.3)
Requirement already satisfied: toolz~=0.10 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.12.1)
Requirement already satisfied: typing-extensions~=4.0 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (4.15.0)
Requirement already satisfied: filelock in /usr/local/lib/python3.12/dist-packages (from huggingface_hub<1.0->huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (3.20.3)
Requirement already satisfied: hf-xet<2.0.0,>=1.1.3 in /usr/local/lib/python3.12/dist-packages (from huggingface_hub<1.0->huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (1.2.0)
Requirement already satisfied: future in /usr/local/lib/python3.12/dist-packages (from hyperopt<0.2.8,>=0.2.7->autogluon.core[all]==1.5.0->autogluon) (1.0.0)
Requirement already satisfied: py4j in /usr/local/lib/python3.12/dist-packages (from hyperopt<0.2.8,>=0.2.7->autogluon.core[all]==1.5.0->autogluon) (0.10.9.9)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.12/dist-packages (from jinja2<3.2,>=3.0.3->autogluon.multimodal==1.5.0->autogluon) (3.0.3)
Requirement already satisfied: attrs>=22.2.0 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (25.4.0)
Requirement already satisfied: jsonschema-specifications>=2023.03.6 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (2025.9.1)
Requirement already satisfied: referencing>=0.28.4 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (0.37.0)
Requirement already satisfied: rpds-py>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (0.30.0)
Requirement already satisfied: lightning-utilities<2.0,>=0.10.0 in /usr/local/lib/python3.12/dist-packages (from lightning<2.6,>=2.5.1->autogluon.multimodal==1.5.0->autogluon) (0.15.2)
Requirement already satisfied: pytorch-lightning in /usr/local/lib/python3.12/dist-packages (from lightning<2.6,>=2.5.1->autogluon.multimodal==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: contourpy>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.3.3)
Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (0.12.1)
Requirement already satisfied: fonttools>=4.22.0 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (4.61.1)
Requirement already satisfied: kiwisolver>=1.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.4.9)
Requirement already satisfied: pyparsing>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.3.2)
Requirement already satisfied: python-dateutil>=2.7 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.9.0.post0)
Requirement already satisfied: numba in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.60.0)
Requirement already satisfied: optuna in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (4.7.0)
Requirement already satisfied: window-ops in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.0.15)
Requirement already satisfied: gdown>=4.0.0 in /usr/local/lib/python3.12/dist-packages (from nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (5.2.1)
Requirement already satisfied: click in /usr/local/lib/python3.12/dist-packages (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon) (8.2.1)
Requirement already satisfied: regex>=2021.8.3 in /usr/local/lib/python3.12/dist-packages (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon) (2025.11.3)
Requirement already satisfied: antlr4-python3-runtime==4.9.* in /usr/local/lib/python3.12/dist-packages (from omegaconf<2.4.0,>=2.1.1->autogluon.multimodal==1.5.0->autogluon) (4.9.3)
Requirement already satisfied: colorama in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.4.6)
Requirement already satisfied: model-index in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.1.11)
Requirement already satisfied: opendatalab in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.0.10)
Requirement already satisfied: rich in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (13.9.4)
Requirement already satisfied: tabulate in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.9.0)
Requirement already satisfied: pytz>=2020.1 in /usr/local/lib/python3.12/dist-packages (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2025.2)
Requirement already satisfied: tzdata>=2022.7 in /usr/local/lib/python3.12/dist-packages (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2025.3)
Requirement already satisfied: msgpack<2.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from ray<2.53,>=2.43.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.1.2)
Requirement already satisfied: protobuf>=3.20.3 in /usr/local/lib/python3.12/dist-packages (from ray<2.53,>=2.43.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (5.29.5)
Requirement already satisfied: aiohttp_cors in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.8.1)
Requirement already satisfied: colorful in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.5.8)
Requirement already satisfied: py-spy>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.1)
Requirement already satisfied: grpcio>=1.42.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.76.0)
Requirement already satisfied: opencensus in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.11.4)
Requirement already satisfied: opentelemetry-sdk>=1.30.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: opentelemetry-exporter-prometheus in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.58b0)
Requirement already satisfied: opentelemetry-proto in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: prometheus_client>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.24.1)
Requirement already satisfied: smart_open in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (7.5.0)
Requirement already satisfied: virtualenv!=20.21.1,>=20.0.24 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (20.36.1)
Requirement already satisfied: tensorboardX>=1.9 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.6.4)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.4.4)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.11)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2026.1.4)
Requirement already satisfied: imageio!=2.35.0,>=2.33 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (2.37.2)
Requirement already satisfied: tifffile>=2022.8.12 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (2026.1.14)
Requirement already satisfied: lazy-loader>=0.4 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (0.4)
Requirement already satisfied: threadpoolctl>=3.1.0 in /usr/local/lib/python3.12/dist-packages (from scikit-learn<1.8.0,>=1.4.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.6.0)
Requirement already satisfied: spacy-legacy<3.1.0,>=3.0.11 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (3.0.12)
Requirement already satisfied: spacy-loggers<2.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.0.5)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.0.15)
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.0.13)
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (3.0.12)
Requirement already satisfied: thinc<8.4.0,>=8.3.4 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (8.3.10)
Requirement already satisfied: wasabi<1.2.0,>=0.9.1 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.1.3)
Requirement already satisfied: srsly<3.0.0,>=2.4.3 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.5.2)
Requirement already satisfied: catalogue<2.1.0,>=2.0.6 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.0.10)
Requirement already satisfied: weasel<0.5.0,>=0.4.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.4.3)
Requirement already satisfied: typer-slim<1.0.0,>=0.3.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.21.1)
Requirement already satisfied: setuptools in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (75.2.0)
Requirement already satisfied: statsmodels>=0.13.2 in /usr/local/lib/python3.12/dist-packages (from statsforecast<2.0.2,>=1.7.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.14.6)
Requirement already satisfied: pbr>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from stevedore<5.5->autogluon.core[all]==1.5.0->autogluon) (7.0.3)
Requirement already satisfied: absl-py>=0.4 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (1.4.0)
Requirement already satisfied: markdown>=2.6.8 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (3.10.1)
Requirement already satisfied: tensorboard-data-server<0.8.0,>=0.7.0 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (0.7.2)
Requirement already satisfied: werkzeug>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (3.1.5)
Requirement already satisfied: sympy>=1.13.3 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.14.0)
Requirement already satisfied: nvidia-cuda-nvrtc-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-cuda-runtime-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-cuda-cupti-cu12==12.6.80 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.80)
Requirement already satisfied: nvidia-cudnn-cu12==9.10.2.21 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (9.10.2.21)
Requirement already satisfied: nvidia-cublas-cu12==12.6.4.1 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.4.1)
Requirement already satisfied: nvidia-cufft-cu12==11.3.0.4 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (11.3.0.4)
Requirement already satisfied: nvidia-curand-cu12==10.3.7.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (10.3.7.77)
Requirement already satisfied: nvidia-cusolver-cu12==11.7.1.2 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (11.7.1.2)
Requirement already satisfied: nvidia-cusparse-cu12==12.5.4.2 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.5.4.2)
Requirement already satisfied: nvidia-cusparselt-cu12==0.7.1 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: nvidia-nccl-cu12==2.27.5 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (2.27.5)
Requirement already satisfied: nvidia-nvshmem-cu12==3.3.20 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (3.3.20)
Requirement already satisfied: nvidia-nvtx-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-nvjitlink-cu12==12.6.85 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.85)
Requirement already satisfied: nvidia-cufile-cu12==1.11.1.6 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.11.1.6)
Requirement already satisfied: triton==3.5.0 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (3.5.0)
Requirement already satisfied: tokenizers<=0.23.0,>=0.22.0 in /usr/local/lib/python3.12/dist-packages (from transformers<4.58,>=4.51.0->transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (0.22.2)
Requirement already satisfied: sentencepiece!=0.1.92,>=0.1.91 in /usr/local/lib/python3.12/dist-packages (from transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (0.2.1)
Requirement already satisfied: frozendict in /usr/local/lib/python3.12/dist-packages (from einx->autogluon.tabular[all]==1.5.0->autogluon) (2.4.7)
Requirement already satisfied: aiohappyeyeballs>=2.5.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: aiosignal>=1.4.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.4.0)
Requirement already satisfied: frozenlist>=1.1.1 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.8.0)
Requirement already satisfied: multidict<7.0,>=4.5 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (6.7.1)
Requirement already satisfied: propcache>=0.2.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (0.4.1)
Requirement already satisfied: yarl<2.0,>=1.17.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.22.0)
Requirement already satisfied: python-fasthtml>=0.12.34 in /usr/local/lib/python3.12/dist-packages (from fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.12.39)
Requirement already satisfied: beautifulsoup4 in /usr/local/lib/python3.12/dist-packages (from gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (4.13.5)
Requirement already satisfied: llvmlite<0.44,>=0.43.0dev0 in /usr/local/lib/python3.12/dist-packages (from numba->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.43.0)
Requirement already satisfied: opentelemetry-api==1.37.0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: opentelemetry-semantic-conventions==0.58b0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.58b0)
Requirement already satisfied: importlib-metadata<8.8.0,>=6.0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-api==1.37.0->opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (8.7.1)
Requirement already satisfied: annotated-types>=0.6.0 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.7.0)
Requirement already satisfied: pydantic-core==2.41.4 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.41.4)
Requirement already satisfied: typing-inspection>=0.4.2 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.4.2)
Requirement already satisfied: patsy>=0.5.6 in /usr/local/lib/python3.12/dist-packages (from statsmodels>=0.13.2->statsforecast<2.0.2,>=1.7.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.0.2)
Requirement already satisfied: mpmath<1.4,>=1.1.0 in /usr/local/lib/python3.12/dist-packages (from sympy>=1.13.3->torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.3.0)
Requirement already satisfied: blis<1.4.0,>=1.3.0 in /usr/local/lib/python3.12/dist-packages (from thinc<8.4.0,>=8.3.4->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.3.3)
Requirement already satisfied: confection<1.0.0,>=0.0.1 in /usr/local/lib/python3.12/dist-packages (from thinc<8.4.0,>=8.3.4->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.1.5)
Requirement already satisfied: distlib<1,>=0.3.7 in /usr/local/lib/python3.12/dist-packages (from virtualenv!=20.21.1,>=20.0.24->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.0)
Requirement already satisfied: platformdirs<5,>=3.9.1 in /usr/local/lib/python3.12/dist-packages (from virtualenv!=20.21.1,>=20.0.24->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (4.5.1)
Requirement already satisfied: cloudpathlib<1.0.0,>=0.7.0 in /usr/local/lib/python3.12/dist-packages (from weasel<0.5.0,>=0.4.2->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.23.0)
Requirement already satisfied: wrapt in /usr/local/lib/python3.12/dist-packages (from smart_open->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.0.1)
Requirement already satisfied: ordered-set in /usr/local/lib/python3.12/dist-packages (from model-index->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (4.1.0)
Requirement already satisfied: opencensus-context>=0.1.3 in /usr/local/lib/python3.12/dist-packages (from opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.1.3)
Requirement already satisfied: google-api-core<3.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.29.0)
Requirement already satisfied: pycryptodome in /usr/local/lib/python3.12/dist-packages (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (3.23.0)
Requirement already satisfied: openxlab in /usr/local/lib/python3.12/dist-packages (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.0.11)
Requirement already satisfied: alembic>=1.5.0 in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.18.1)
Requirement already satisfied: colorlog in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (6.10.1)
Requirement already satisfied: sqlalchemy>=1.4.2 in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.0.46)
Requirement already satisfied: tenacity>=6.2.0 in /usr/local/lib/python3.12/dist-packages (from plotly->catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (9.1.2)
Requirement already satisfied: beartype>=0.16.2 in /usr/local/lib/python3.12/dist-packages (from plum-dispatch->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.22.9)
Requirement already satisfied: markdown-it-py>=2.2.0 in /usr/local/lib/python3.12/dist-packages (from rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (4.0.0)
Requirement already satisfied: pygments<3.0.0,>=2.13.0 in /usr/local/lib/python3.12/dist-packages (from rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (2.19.2)
Requirement already satisfied: Mako in /usr/local/lib/python3.12/dist-packages (from alembic>=1.5.0->optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.3.10)
Requirement already satisfied: googleapis-common-protos<2.0.0,>=1.56.2 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.72.0)
Requirement already satisfied: proto-plus<2.0.0,>=1.22.3 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.27.0)
Requirement already satisfied: google-auth<3.0.0,>=2.14.1 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.47.0)
Requirement already satisfied: mdurl~=0.1 in /usr/local/lib/python3.12/dist-packages (from markdown-it-py>=2.2.0->rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.1.2)
Requirement already satisfied: starlette>0.33 in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.50.0)
Requirement already satisfied: oauthlib in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.3.1)
Requirement already satisfied: itsdangerous in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (2.2.0)
Requirement already satisfied: uvicorn>=0.30 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.40.0)
Requirement already satisfied: httpx in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.28.1)
Requirement already satisfied: fastlite>=0.1.1 in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.2.4)
Requirement already satisfied: python-multipart in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.22)
Requirement already satisfied: greenlet>=1 in /usr/local/lib/python3.12/dist-packages (from sqlalchemy>=1.4.2->optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (3.3.1)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.12/dist-packages (from beautifulsoup4->gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (2.8.3)
Requirement already satisfied: PySocks!=1.5.7,>=1.5.6 in /usr/local/lib/python3.12/dist-packages (from requests[socks]->gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (1.7.1)
Requirement already satisfied: apswutils>=0.1.2 in /usr/local/lib/python3.12/dist-packages (from fastlite>=0.1.1->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.1.2)
Requirement already satisfied: pyasn1-modules>=0.2.1 in /usr/local/lib/python3.12/dist-packages (from google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.2)
Requirement already satisfied: rsa<5,>=3.1.4 in /usr/local/lib/python3.12/dist-packages (from google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (4.9.1)
Requirement already satisfied: zipp>=3.20 in /usr/local/lib/python3.12/dist-packages (from importlib-metadata<8.8.0,>=6.0->opentelemetry-api==1.37.0->opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (3.23.0)
Requirement already satisfied: anyio<5,>=3.6.2 in /usr/local/lib/python3.12/dist-packages (from starlette>0.33->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (4.12.1)
Requirement already satisfied: h11>=0.8 in /usr/local/lib/python3.12/dist-packages (from uvicorn>=0.30->uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.16.0)
Requirement already satisfied: httptools>=0.6.3 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: python-dotenv>=0.13 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.2.1)
Requirement already satisfied: uvloop>=0.15.1 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.22.1)
Requirement already satisfied: watchfiles>=0.13 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.1.1)
Requirement already satisfied: websockets>=10.4 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (15.0.1)
Requirement already satisfied: httpcore==1.* in /usr/local/lib/python3.12/dist-packages (from httpx->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.0.9)
Requirement already satisfied: apsw in /usr/local/lib/python3.12/dist-packages (from apswutils>=0.1.2->fastlite>=0.1.1->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.51.2.0)
Requirement already satisfied: pyasn1<0.7.0,>=0.6.1 in /usr/local/lib/python3.12/dist-packages (from pyasn1-modules>=0.2.1->google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.6.2)
Creating CSV from Arrow format: 100% 4/4 [00:01<00:00,  3.00ba/s]Creating CSV from Arrow format: 100% 1/1 [00:00<00:00, 12.06ba/s]Downloads complete
No path specified. Models will be saved in: "AutogluonModels/ag-20260201_235048"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cu126
CUDA Version:       12.6
GPU Memory:         GPU 0: 14.74/14.74 GB
Total GPU Memory:   Free: 14.74 GB, Allocated: 0.00 GB, Total: 14.74 GB
GPU Count:          1
Memory Avail:       5.65 GB / 12.67 GB (44.6%)
Disk Space Avail:   72.76 GB / 112.64 GB (64.6%)
===================================================
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme'  : New in v1.5: The state-of-the-art for tabular data. Massively better than 'best' on datasets <100000 samples by using new Tabular Foundation Models (TFMs) meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, TabDPT, and TabM. Requires a GPU and `pip install autogluon.tabular[tabarena]` to install TabPFN, TabICL, and TabDPT.
	presets='best'     : Maximize accuracy. Recommended for most users. Use in competitions and benchmarks.
	presets='best_v150': New in v1.5: Better quality than 'best' and 5x+ faster to train. Give it a try!
	presets='high'     : Strong accuracy with fast inference speed.
	presets='high_v150': New in v1.5: Better quality than 'high' and 5x+ faster to train. Give it a try!
	presets='good'     : Good accuracy with very fast inference speed.
	presets='medium'   : Fast training time, ideal for initial prototyping.
Using hyperparameters preset: hyperparameters='default'
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "/content/AutogluonModels/ag-20260201_235048"
Train Data Rows:    3448
Train Data Columns: 3
Label Column:       clarity_label
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['Clear Reply', 'Ambivalent', 'Clear Non-Reply']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    5749.55 MB
	Train Data (Original)  Memory Usage: 10.16 MB (0.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['interview_question', 'interview_answer', 'question']
			CountVectorizer fit with vocabulary size = 8559
		Warning: Due to memory constraints, ngram feature count is being reduced. Allocate more memory to maximize model quality.
		Reducing Vectorizer vocab size from 8559 to 3955 to avoid OOM error
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('object', ['text']) : 3 | ['interview_question', 'interview_answer', 'question']
	Types of features in processed data (raw dtype, special dtypes):
		('category', ['text_as_category'])  :    3 | ['interview_question', 'interview_answer', 'question']
		('int', ['binned', 'text_special']) :   57 | ['interview_question.char_count', 'interview_question.word_count', 'interview_question.capital_ratio', 'interview_question.lower_ratio', 'interview_question.digit_ratio', ...]
		('int', ['text_ngram'])             : 3926 | ['__nlp__.000', '__nlp__.10', '__nlp__.100', '__nlp__.11', '__nlp__.12', ...]
	44.4s = Fit runtime
	3 features in original data used to generate 3986 features in processed data.
	Train Data (Processed) Memory Usage: 26.03 MB (0.5% of available memory)
Data preprocessing and feature engineering runtime = 45.7s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To change this, specify the eval_metric parameter of Predictor()
Automatically generating train/validation split with holdout_frac=0.145, Train Rows: 2948, Val Rows: 500
User-specified model hyperparameters to be fit:
{
	'NN_TORCH': [{}],
	'GBM': [{'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}}, {}, {'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'ag_args': {'name_suffix': 'Large', 'priority': 0, 'hyperparameter_tune_kwargs': None}}],
	'CAT': [{}],
	'XGB': [{}],
	'FASTAI': [{}],
	'RF': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
	'XT': [{'criterion': 'gini', 'ag_args': {'name_suffix': 'Gini', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'entropy', 'ag_args': {'name_suffix': 'Entr', 'problem_types': ['binary', 'multiclass']}}, {'criterion': 'squared_error', 'ag_args': {'name_suffix': 'MSE', 'problem_types': ['regression', 'quantile']}}],
}
Fitting 11 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI ... Training model for up to 14.30s of the 14.29s of remaining time.
	Fitting with cpus=1, gpus=0, mem=0.2/5.5 GB
	0.608	 = Validation score   (accuracy)
	11.59s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: LightGBMXT ... Training model for up to 2.66s of the 2.65s of remaining time.
	Fitting with cpus=1, gpus=0, mem=0.9/6.4 GB
	Ran out of time, early stopping on iteration 1. Best iteration is:
	[1]	valid_set's multi_error: 0.408
	0.592	 = Validation score   (accuracy)
	6.02s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ... Training model for up to 14.30s of the -3.46s of remaining time.
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=0.0/6.3 GB
	Ensemble Weights: {'NeuralNetFastAI': 0.75, 'LightGBMXT': 0.25}
	0.61	 = Validation score   (accuracy)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 63.73s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 7062.0 rows/s (500 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20260201_235048")
Loaded data from: evaluation.csv | Columns = 20 / 20 | Rows = 237 -> 237
```

## Autogluon with preset high_v150 and time limit 20 minutes

```
 Requirement already satisfied: autogluon in /usr/local/lib/python3.12/dist-packages (1.5.0)
Requirement already satisfied: autogluon.core==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: autogluon.features==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon) (1.5.0)
Requirement already satisfied: autogluon.tabular==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: autogluon.multimodal==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon) (1.5.0)
Requirement already satisfied: autogluon.timeseries==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: numpy<2.4.0,>=1.25.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.0.2)
Requirement already satisfied: scipy<1.17,>=1.5.4 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.16.3)
Requirement already satisfied: scikit-learn<1.8.0,>=1.4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.6.1)
Requirement already satisfied: networkx<4,>=3.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.6.1)
Requirement already satisfied: pandas<2.4.0,>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.2.2)
Requirement already satisfied: tqdm<5,>=4.38 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (4.67.1)
Requirement already satisfied: requests in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.32.4)
Requirement already satisfied: matplotlib<3.11,>=3.7.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.10.0)
Requirement already satisfied: boto3<2,>=1.10 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.42.39)
Requirement already satisfied: autogluon.common==1.5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.5.0)
Requirement already satisfied: ray<2.53,>=2.43.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.52.1)
Requirement already satisfied: pyarrow>=15.0.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (18.1.0)
Requirement already satisfied: hyperopt<0.2.8,>=0.2.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (0.2.7)
Requirement already satisfied: stevedore<5.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (5.4.1)
Requirement already satisfied: Pillow<12,>=10.0.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (11.3.0)
Requirement already satisfied: torch<2.10,>=2.6 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.9.0+cu126)
Requirement already satisfied: lightning<2.6,>=2.5.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.5.6)
Requirement already satisfied: transformers<4.58,>=4.51.0 in /usr/local/lib/python3.12/dist-packages (from transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (4.57.6)
Requirement already satisfied: accelerate<2.0,>=0.34.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.12.0)
Requirement already satisfied: fsspec<=2025.3 in /usr/local/lib/python3.12/dist-packages (from fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (2025.3.0)
Requirement already satisfied: jsonschema<4.24,>=4.18 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (4.23.0)
Requirement already satisfied: seqeval<1.3.0,>=1.2.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.2.2)
Requirement already satisfied: evaluate<0.5.0,>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.4.6)
Requirement already satisfied: timm<1.0.7,>=0.9.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.0.3)
Requirement already satisfied: torchvision<0.25.0,>=0.21.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.24.0+cu126)
Requirement already satisfied: scikit-image<0.26.0,>=0.19.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.25.2)
Requirement already satisfied: text-unidecode<1.4,>=1.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.3)
Requirement already satisfied: torchmetrics<1.8,>=1.2.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.7.4)
Requirement already satisfied: omegaconf<2.4.0,>=2.1.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.3.0)
Requirement already satisfied: pytorch-metric-learning<2.9,>=1.3.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.8.1)
Requirement already satisfied: nlpaug<1.2.0,>=1.1.10 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.1.11)
Requirement already satisfied: nltk<3.10,>=3.4.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (3.9.1)
Requirement already satisfied: openmim<0.4.0,>=0.3.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.3.9)
Requirement already satisfied: defusedxml<0.7.2,>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: jinja2<3.2,>=3.0.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (3.1.6)
Requirement already satisfied: tensorboard<3,>=2.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.19.0)
Requirement already satisfied: pytesseract<0.4,>=0.3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.3.13)
Requirement already satisfied: nvidia-ml-py3<8.0,>=7.352.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (7.352.0)
Requirement already satisfied: pdf2image<1.19,>=1.17.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.17.0)
Requirement already satisfied: einx in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.3.0)
Requirement already satisfied: lightgbm<4.7,>=4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (4.6.0)
Requirement already satisfied: huggingface_hub<1.0 in /usr/local/lib/python3.12/dist-packages (from huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (0.36.0)
Requirement already satisfied: xgboost<3.2,>=2.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (3.1.3)
Requirement already satisfied: catboost<1.3,>=1.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (1.2.8)
Requirement already satisfied: fastai<2.9,>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (2.8.6)
Requirement already satisfied: loguru in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.7.3)
Requirement already satisfied: einops<0.9,>=0.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.8.2)
Requirement already satisfied: spacy<3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (3.8.11)
Requirement already satisfied: joblib<1.7,>=1.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.5.3)
Requirement already satisfied: gluonts<0.17,>=0.15.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.16.2)
Requirement already satisfied: statsforecast<2.0.2,>=1.7.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.0.1)
Requirement already satisfied: mlforecast<0.15.0,>=0.14.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.14.0)
Requirement already satisfied: utilsforecast<0.2.12,>=0.2.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.2.11)
Requirement already satisfied: coreforecast<0.0.17,>=0.0.12 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.0.16)
Requirement already satisfied: fugue>=0.9.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.9.6)
Requirement already satisfied: orjson~=3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (3.11.5)
Requirement already satisfied: chronos-forecasting<2.4,>=2.2.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.2.2)
Requirement already satisfied: peft<0.18,>=0.13.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.17.1)
Requirement already satisfied: psutil<7.2.0,>=5.7.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.common==1.5.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (5.9.5)
Requirement already satisfied: pyyaml>=5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.common==1.5.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (6.0.3)
Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.12/dist-packages (from accelerate<2.0,>=0.34.0->autogluon.multimodal==1.5.0->autogluon) (25.0)
Requirement already satisfied: safetensors>=0.4.3 in /usr/local/lib/python3.12/dist-packages (from accelerate<2.0,>=0.34.0->autogluon.multimodal==1.5.0->autogluon) (0.7.0)
Requirement already satisfied: botocore<1.43.0,>=1.42.39 in /usr/local/lib/python3.12/dist-packages (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.42.39)
Requirement already satisfied: jmespath<2.0.0,>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.1.0)
Requirement already satisfied: s3transfer<0.17.0,>=0.16.0 in /usr/local/lib/python3.12/dist-packages (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (0.16.0)
Requirement already satisfied: graphviz in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (0.21)
Requirement already satisfied: plotly in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (5.24.1)
Requirement already satisfied: six in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (1.17.0)
Requirement already satisfied: datasets>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (4.0.0)
Requirement already satisfied: dill in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (0.3.8)
Requirement already satisfied: xxhash in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (3.6.0)
Requirement already satisfied: multiprocess in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (0.70.16)
Requirement already satisfied: pip in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (24.1.2)
Requirement already satisfied: fastdownload<2,>=0.0.5 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.7)
Requirement already satisfied: fastcore>=1.8.0 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.12.7)
Requirement already satisfied: fasttransform>=0.0.2 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.2)
Requirement already satisfied: fastprogress>=0.2.4 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.1.3)
Requirement already satisfied: plum-dispatch in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: cloudpickle in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.1.2)
Requirement already satisfied: aiohttp!=4.0.0a0,!=4.0.0a1 in /usr/local/lib/python3.12/dist-packages (from fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (3.13.3)
Requirement already satisfied: triad>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from fugue>=0.9.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.0.0)
Requirement already satisfied: adagio>=0.2.6 in /usr/local/lib/python3.12/dist-packages (from fugue>=0.9.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.2.6)
Requirement already satisfied: pydantic<3,>=1.7 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.12.3)
Requirement already satisfied: toolz~=0.10 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.12.1)
Requirement already satisfied: typing-extensions~=4.0 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (4.15.0)
Requirement already satisfied: filelock in /usr/local/lib/python3.12/dist-packages (from huggingface_hub<1.0->huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (3.20.3)
Requirement already satisfied: hf-xet<2.0.0,>=1.1.3 in /usr/local/lib/python3.12/dist-packages (from huggingface_hub<1.0->huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (1.2.0)
Requirement already satisfied: future in /usr/local/lib/python3.12/dist-packages (from hyperopt<0.2.8,>=0.2.7->autogluon.core[all]==1.5.0->autogluon) (1.0.0)
Requirement already satisfied: py4j in /usr/local/lib/python3.12/dist-packages (from hyperopt<0.2.8,>=0.2.7->autogluon.core[all]==1.5.0->autogluon) (0.10.9.9)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.12/dist-packages (from jinja2<3.2,>=3.0.3->autogluon.multimodal==1.5.0->autogluon) (3.0.3)
Requirement already satisfied: attrs>=22.2.0 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (25.4.0)
Requirement already satisfied: jsonschema-specifications>=2023.03.6 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (2025.9.1)
Requirement already satisfied: referencing>=0.28.4 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (0.37.0)
Requirement already satisfied: rpds-py>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (0.30.0)
Requirement already satisfied: lightning-utilities<2.0,>=0.10.0 in /usr/local/lib/python3.12/dist-packages (from lightning<2.6,>=2.5.1->autogluon.multimodal==1.5.0->autogluon) (0.15.2)
Requirement already satisfied: pytorch-lightning in /usr/local/lib/python3.12/dist-packages (from lightning<2.6,>=2.5.1->autogluon.multimodal==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: contourpy>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.3.3)
Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (0.12.1)
Requirement already satisfied: fonttools>=4.22.0 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (4.61.1)
Requirement already satisfied: kiwisolver>=1.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.4.9)
Requirement already satisfied: pyparsing>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.3.2)
Requirement already satisfied: python-dateutil>=2.7 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.9.0.post0)
Requirement already satisfied: numba in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.60.0)
Requirement already satisfied: optuna in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (4.7.0)
Requirement already satisfied: window-ops in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.0.15)
Requirement already satisfied: gdown>=4.0.0 in /usr/local/lib/python3.12/dist-packages (from nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (5.2.1)
Requirement already satisfied: click in /usr/local/lib/python3.12/dist-packages (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon) (8.2.1)
Requirement already satisfied: regex>=2021.8.3 in /usr/local/lib/python3.12/dist-packages (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon) (2025.11.3)
Requirement already satisfied: antlr4-python3-runtime==4.9.* in /usr/local/lib/python3.12/dist-packages (from omegaconf<2.4.0,>=2.1.1->autogluon.multimodal==1.5.0->autogluon) (4.9.3)
Requirement already satisfied: colorama in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.4.6)
Requirement already satisfied: model-index in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.1.11)
Requirement already satisfied: opendatalab in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.0.10)
Requirement already satisfied: rich in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (13.9.4)
Requirement already satisfied: tabulate in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.9.0)
Requirement already satisfied: pytz>=2020.1 in /usr/local/lib/python3.12/dist-packages (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2025.2)
Requirement already satisfied: tzdata>=2022.7 in /usr/local/lib/python3.12/dist-packages (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2025.3)
Requirement already satisfied: msgpack<2.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from ray<2.53,>=2.43.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.1.2)
Requirement already satisfied: protobuf>=3.20.3 in /usr/local/lib/python3.12/dist-packages (from ray<2.53,>=2.43.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (5.29.5)
Requirement already satisfied: aiohttp_cors in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.8.1)
Requirement already satisfied: colorful in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.5.8)
Requirement already satisfied: py-spy>=0.4.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.1)
Requirement already satisfied: grpcio>=1.42.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.76.0)
Requirement already satisfied: opencensus in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.11.4)
Requirement already satisfied: opentelemetry-sdk>=1.30.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: opentelemetry-exporter-prometheus in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.58b0)
Requirement already satisfied: opentelemetry-proto in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: prometheus_client>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.24.1)
Requirement already satisfied: smart_open in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (7.5.0)
Requirement already satisfied: virtualenv!=20.21.1,>=20.0.24 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (20.36.1)
Requirement already satisfied: tensorboardX>=1.9 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.6.4)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.4.4)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.11)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2026.1.4)
Requirement already satisfied: imageio!=2.35.0,>=2.33 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (2.37.2)
Requirement already satisfied: tifffile>=2022.8.12 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (2026.1.14)
Requirement already satisfied: lazy-loader>=0.4 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (0.4)
Requirement already satisfied: threadpoolctl>=3.1.0 in /usr/local/lib/python3.12/dist-packages (from scikit-learn<1.8.0,>=1.4.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.6.0)
Requirement already satisfied: spacy-legacy<3.1.0,>=3.0.11 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (3.0.12)
Requirement already satisfied: spacy-loggers<2.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.0.5)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.0.15)
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.0.13)
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (3.0.12)
Requirement already satisfied: thinc<8.4.0,>=8.3.4 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (8.3.10)
Requirement already satisfied: wasabi<1.2.0,>=0.9.1 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.1.3)
Requirement already satisfied: srsly<3.0.0,>=2.4.3 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.5.2)
Requirement already satisfied: catalogue<2.1.0,>=2.0.6 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.0.10)
Requirement already satisfied: weasel<0.5.0,>=0.4.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.4.3)
Requirement already satisfied: typer-slim<1.0.0,>=0.3.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.21.1)
Requirement already satisfied: setuptools in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (75.2.0)
Requirement already satisfied: statsmodels>=0.13.2 in /usr/local/lib/python3.12/dist-packages (from statsforecast<2.0.2,>=1.7.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.14.6)
Requirement already satisfied: pbr>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from stevedore<5.5->autogluon.core[all]==1.5.0->autogluon) (7.0.3)
Requirement already satisfied: absl-py>=0.4 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (1.4.0)
Requirement already satisfied: markdown>=2.6.8 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (3.10.1)
Requirement already satisfied: tensorboard-data-server<0.8.0,>=0.7.0 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (0.7.2)
Requirement already satisfied: werkzeug>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (3.1.5)
Requirement already satisfied: sympy>=1.13.3 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.14.0)
Requirement already satisfied: nvidia-cuda-nvrtc-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-cuda-runtime-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-cuda-cupti-cu12==12.6.80 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.80)
Requirement already satisfied: nvidia-cudnn-cu12==9.10.2.21 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (9.10.2.21)
Requirement already satisfied: nvidia-cublas-cu12==12.6.4.1 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.4.1)
Requirement already satisfied: nvidia-cufft-cu12==11.3.0.4 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (11.3.0.4)
Requirement already satisfied: nvidia-curand-cu12==10.3.7.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (10.3.7.77)
Requirement already satisfied: nvidia-cusolver-cu12==11.7.1.2 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (11.7.1.2)
Requirement already satisfied: nvidia-cusparse-cu12==12.5.4.2 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.5.4.2)
Requirement already satisfied: nvidia-cusparselt-cu12==0.7.1 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: nvidia-nccl-cu12==2.27.5 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (2.27.5)
Requirement already satisfied: nvidia-nvshmem-cu12==3.3.20 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (3.3.20)
Requirement already satisfied: nvidia-nvtx-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-nvjitlink-cu12==12.6.85 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.85)
Requirement already satisfied: nvidia-cufile-cu12==1.11.1.6 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.11.1.6)
Requirement already satisfied: triton==3.5.0 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (3.5.0)
Requirement already satisfied: tokenizers<=0.23.0,>=0.22.0 in /usr/local/lib/python3.12/dist-packages (from transformers<4.58,>=4.51.0->transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (0.22.2)
Requirement already satisfied: sentencepiece!=0.1.92,>=0.1.91 in /usr/local/lib/python3.12/dist-packages (from transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (0.2.1)
Requirement already satisfied: frozendict in /usr/local/lib/python3.12/dist-packages (from einx->autogluon.tabular[all]==1.5.0->autogluon) (2.4.7)
Requirement already satisfied: aiohappyeyeballs>=2.5.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: aiosignal>=1.4.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.4.0)
Requirement already satisfied: frozenlist>=1.1.1 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.8.0)
Requirement already satisfied: multidict<7.0,>=4.5 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (6.7.1)
Requirement already satisfied: propcache>=0.2.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (0.4.1)
Requirement already satisfied: yarl<2.0,>=1.17.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.22.0)
Requirement already satisfied: python-fasthtml>=0.12.34 in /usr/local/lib/python3.12/dist-packages (from fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.12.39)
Requirement already satisfied: beautifulsoup4 in /usr/local/lib/python3.12/dist-packages (from gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (4.13.5)
Requirement already satisfied: llvmlite<0.44,>=0.43.0dev0 in /usr/local/lib/python3.12/dist-packages (from numba->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.43.0)
Requirement already satisfied: opentelemetry-api==1.37.0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: opentelemetry-semantic-conventions==0.58b0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.58b0)
Requirement already satisfied: importlib-metadata<8.8.0,>=6.0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-api==1.37.0->opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (8.7.1)
Requirement already satisfied: annotated-types>=0.6.0 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.7.0)
Requirement already satisfied: pydantic-core==2.41.4 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.41.4)
Requirement already satisfied: typing-inspection>=0.4.2 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.4.2)
Requirement already satisfied: patsy>=0.5.6 in /usr/local/lib/python3.12/dist-packages (from statsmodels>=0.13.2->statsforecast<2.0.2,>=1.7.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.0.2)
Requirement already satisfied: mpmath<1.4,>=1.1.0 in /usr/local/lib/python3.12/dist-packages (from sympy>=1.13.3->torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.3.0)
Requirement already satisfied: blis<1.4.0,>=1.3.0 in /usr/local/lib/python3.12/dist-packages (from thinc<8.4.0,>=8.3.4->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.3.3)
Requirement already satisfied: confection<1.0.0,>=0.0.1 in /usr/local/lib/python3.12/dist-packages (from thinc<8.4.0,>=8.3.4->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.1.5)
Requirement already satisfied: distlib<1,>=0.3.7 in /usr/local/lib/python3.12/dist-packages (from virtualenv!=20.21.1,>=20.0.24->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.0)
Requirement already satisfied: platformdirs<5,>=3.9.1 in /usr/local/lib/python3.12/dist-packages (from virtualenv!=20.21.1,>=20.0.24->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (4.5.1)
Requirement already satisfied: cloudpathlib<1.0.0,>=0.7.0 in /usr/local/lib/python3.12/dist-packages (from weasel<0.5.0,>=0.4.2->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.23.0)
Requirement already satisfied: wrapt in /usr/local/lib/python3.12/dist-packages (from smart_open->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.0.1)
Requirement already satisfied: ordered-set in /usr/local/lib/python3.12/dist-packages (from model-index->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (4.1.0)
Requirement already satisfied: opencensus-context>=0.1.3 in /usr/local/lib/python3.12/dist-packages (from opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.1.3)
Requirement already satisfied: google-api-core<3.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.29.0)
Requirement already satisfied: pycryptodome in /usr/local/lib/python3.12/dist-packages (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (3.23.0)
Requirement already satisfied: openxlab in /usr/local/lib/python3.12/dist-packages (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.0.11)
Requirement already satisfied: alembic>=1.5.0 in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.18.1)
Requirement already satisfied: colorlog in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (6.10.1)
Requirement already satisfied: sqlalchemy>=1.4.2 in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.0.46)
Requirement already satisfied: tenacity>=6.2.0 in /usr/local/lib/python3.12/dist-packages (from plotly->catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (9.1.2)
Requirement already satisfied: beartype>=0.16.2 in /usr/local/lib/python3.12/dist-packages (from plum-dispatch->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.22.9)
Requirement already satisfied: markdown-it-py>=2.2.0 in /usr/local/lib/python3.12/dist-packages (from rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (4.0.0)
Requirement already satisfied: pygments<3.0.0,>=2.13.0 in /usr/local/lib/python3.12/dist-packages (from rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (2.19.2)
Requirement already satisfied: Mako in /usr/local/lib/python3.12/dist-packages (from alembic>=1.5.0->optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.3.10)
Requirement already satisfied: googleapis-common-protos<2.0.0,>=1.56.2 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.72.0)
Requirement already satisfied: proto-plus<2.0.0,>=1.22.3 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.27.0)
Requirement already satisfied: google-auth<3.0.0,>=2.14.1 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.47.0)
Requirement already satisfied: mdurl~=0.1 in /usr/local/lib/python3.12/dist-packages (from markdown-it-py>=2.2.0->rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.1.2)
Requirement already satisfied: starlette>0.33 in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.50.0)
Requirement already satisfied: oauthlib in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.3.1)
Requirement already satisfied: itsdangerous in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (2.2.0)
Requirement already satisfied: uvicorn>=0.30 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.40.0)
Requirement already satisfied: httpx in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.28.1)
Requirement already satisfied: fastlite>=0.1.1 in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.2.4)
Requirement already satisfied: python-multipart in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.22)
Requirement already satisfied: greenlet>=1 in /usr/local/lib/python3.12/dist-packages (from sqlalchemy>=1.4.2->optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (3.3.1)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.12/dist-packages (from beautifulsoup4->gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (2.8.3)
Requirement already satisfied: PySocks!=1.5.7,>=1.5.6 in /usr/local/lib/python3.12/dist-packages (from requests[socks]->gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (1.7.1)
Requirement already satisfied: apswutils>=0.1.2 in /usr/local/lib/python3.12/dist-packages (from fastlite>=0.1.1->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.1.2)
Requirement already satisfied: pyasn1-modules>=0.2.1 in /usr/local/lib/python3.12/dist-packages (from google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.2)
Requirement already satisfied: rsa<5,>=3.1.4 in /usr/local/lib/python3.12/dist-packages (from google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (4.9.1)
Requirement already satisfied: zipp>=3.20 in /usr/local/lib/python3.12/dist-packages (from importlib-metadata<8.8.0,>=6.0->opentelemetry-api==1.37.0->opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (3.23.0)
Requirement already satisfied: anyio<5,>=3.6.2 in /usr/local/lib/python3.12/dist-packages (from starlette>0.33->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (4.12.1)
Requirement already satisfied: h11>=0.8 in /usr/local/lib/python3.12/dist-packages (from uvicorn>=0.30->uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.16.0)
Requirement already satisfied: httptools>=0.6.3 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: python-dotenv>=0.13 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.2.1)
Requirement already satisfied: uvloop>=0.15.1 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.22.1)
Requirement already satisfied: watchfiles>=0.13 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.1.1)
Requirement already satisfied: websockets>=10.4 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (15.0.1)
Requirement already satisfied: httpcore==1.* in /usr/local/lib/python3.12/dist-packages (from httpx->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.0.9)
Requirement already satisfied: apsw in /usr/local/lib/python3.12/dist-packages (from apswutils>=0.1.2->fastlite>=0.1.1->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.51.2.0)
Requirement already satisfied: pyasn1<0.7.0,>=0.6.1 in /usr/local/lib/python3.12/dist-packages (from pyasn1-modules>=0.2.1->google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.6.2)
Creating CSV from Arrow format: 100% 4/4 [00:01<00:00,  2.53ba/s]Creating CSV from Arrow format: 100% 1/1 [00:00<00:00,  6.80ba/s]Downloads complete
No path specified. Models will be saved in: "AutogluonModels/ag-20260202_001125"
Preset alias specified: 'high_v150' maps to 'high_quality_v150'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cu126
CUDA Version:       12.6
GPU Memory:         GPU 0: 14.74/14.74 GB
Total GPU Memory:   Free: 14.74 GB, Allocated: 0.00 GB, Total: 14.74 GB
GPU Count:          1
Memory Avail:       7.97 GB / 12.67 GB (62.9%)
Disk Space Avail:   72.66 GB / 112.64 GB (64.5%)
===================================================
Presets specified: ['high_v150']
Using hyperparameters preset: hyperparameters='zeroshot_2025_12_18_cpu'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
Beginning AutoGluon training ... Time limit = 1200s
AutoGluon will save models to "/content/AutogluonModels/ag-20260202_001125"
Train Data Rows:    3448
Train Data Columns: 3
Label Column:       clarity_label
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['Clear Reply', 'Ambivalent', 'Clear Non-Reply']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    8087.89 MB
	Train Data (Original)  Memory Usage: 10.16 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['interview_question', 'interview_answer', 'question']
			CountVectorizer fit with vocabulary size = 8559
		Warning: Due to memory constraints, ngram feature count is being reduced. Allocate more memory to maximize model quality.
		Reducing Vectorizer vocab size from 8559 to 4538 to avoid OOM error
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('object', ['text']) : 3 | ['interview_question', 'interview_answer', 'question']
	Types of features in processed data (raw dtype, special dtypes):
		('category', ['text_as_category'])  :    3 | ['interview_question', 'interview_answer', 'question']
		('int', ['binned', 'text_special']) :   57 | ['interview_question.char_count', 'interview_question.word_count', 'interview_question.capital_ratio', 'interview_question.lower_ratio', 'interview_question.digit_ratio', ...]
		('int', ['text_ngram'])             : 4497 | ['__nlp__.000', '__nlp__.10', '__nlp__.100', '__nlp__.11', '__nlp__.12', ...]
	38.0s = Fit runtime
	3 features in original data used to generate 4557 features in processed data.
	Train Data (Processed) Memory Usage: 29.78 MB (0.4% of available memory)
Data preprocessing and feature engineering runtime = 38.5s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To change this, specify the eval_metric parameter of Predictor()
Large model count detected (50 configs) ... Only displaying the first 3 models of each family. To see all, set `verbosity=3`.
User-specified model hyperparameters to be fit:
{
	'CAT': [{'ag_args': {'name_suffix': '_c1', 'priority': -1}}],
	'GBM_PREP': [{'ag.prep_params': [[[['ArithmeticFeatureGenerator', {}]], [['CategoricalInteractionFeatureGenerator', {'passthrough': True}], ['OOFTargetEncodingFeatureGenerator', {}]]]], 'ag.prep_params.passthrough_types': {'invalid_raw_types': ['category', 'object']}, 'ag_args': {'name_suffix': '_r13', 'priority': -2}, 'ag_args_ensemble': {'model_random_seed': 0, 'vary_seed_across_folds': True}, 'bagging_fraction': 0.9923026236907, 'bagging_freq': 1, 'cat_l2': 0.014290368488, 'cat_smooth': 1.8662939903973, 'extra_trees': True, 'feature_fraction': 0.5533919718605, 'lambda_l1': 0.914411672958, 'lambda_l2': 1.90439560009, 'learning_rate': 0.0193225778401, 'max_cat_to_onehot': 18, 'min_data_in_leaf': 28, 'min_data_per_group': 54, 'num_leaves': 64}, {'ag.prep_params': [[[['ArithmeticFeatureGenerator', {}]], [['CategoricalInteractionFeatureGenerator', {'passthrough': True}], ['OOFTargetEncodingFeatureGenerator', {}]]]], 'ag.prep_params.passthrough_types': {'invalid_raw_types': ['category', 'object']}, 'ag_args': {'name_suffix': '_r41', 'priority': -7}, 'ag_args_ensemble': {'model_random_seed': 0, 'vary_seed_across_folds': True}, 'bagging_fraction': 0.7215411996558, 'bagging_freq': 1, 'cat_l2': 1.887369154362, 'cat_smooth': 0.0278693980873, 'extra_trees': True, 'feature_fraction': 0.4247583287144, 'lambda_l1': 0.1129800247772, 'lambda_l2': 0.2623265718536, 'learning_rate': 0.0074201920651, 'max_cat_to_onehot': 9, 'min_data_in_leaf': 15, 'min_data_per_group': 10, 'num_leaves': 8}, {'ag.prep_params': [[[['ArithmeticFeatureGenerator', {}]], [['CategoricalInteractionFeatureGenerator', {'passthrough': True}], ['OOFTargetEncodingFeatureGenerator', {}]]]], 'ag.prep_params.passthrough_types': {'invalid_raw_types': ['category', 'object']}, 'ag_args': {'name_suffix': '_r31', 'priority': -10}, 'ag_args_ensemble': {'model_random_seed': 0, 'vary_seed_across_folds': True}, 'bagging_fraction': 0.9591526242875, 'bagging_freq': 1, 'cat_l2': 1.8962346412823, 'cat_smooth': 0.0215219089995, 'extra_trees': False, 'feature_fraction': 0.5791844062459, 'lambda_l1': 0.938461750637, 'lambda_l2': 0.9899852075056, 'learning_rate': 0.0397613094741, 'max_cat_to_onehot': 27, 'min_data_in_leaf': 1, 'min_data_per_group': 39, 'num_leaves': 16}],
	'GBM': [{'ag_args': {'name_suffix': '_r177', 'priority': -3}, 'bagging_fraction': 0.8769107816033, 'bagging_freq': 1, 'cat_l2': 0.3418014393813, 'cat_smooth': 15.4304556649114, 'extra_trees': True, 'feature_fraction': 0.4622189821941, 'lambda_l1': 0.2375070586896, 'lambda_l2': 0.3551561351804, 'learning_rate': 0.0178593900218, 'max_cat_to_onehot': 16, 'min_data_in_leaf': 3, 'min_data_per_group': 9, 'num_leaves': 39}, {'ag_args': {'name_suffix': '_r163', 'priority': -5}, 'bagging_fraction': 0.9783898288461, 'bagging_freq': 1, 'cat_l2': 0.1553395260142, 'cat_smooth': 0.0093122749318, 'extra_trees': False, 'feature_fraction': 0.5279825611461, 'lambda_l1': 0.0269274915833, 'lambda_l2': 0.8375250972309, 'learning_rate': 0.0113913650333, 'max_cat_to_onehot': 42, 'min_data_in_leaf': 3, 'min_data_per_group': 75, 'num_leaves': 84}, {'ag_args': {'name_suffix': '_r72', 'priority': -8}, 'bagging_fraction': 0.950146543918, 'bagging_freq': 1, 'cat_l2': 0.2159137242663, 'cat_smooth': 0.0638204395719, 'extra_trees': True, 'feature_fraction': 0.4044759649281, 'lambda_l1': 0.7661581500422, 'lambda_l2': 1.6041759693902, 'learning_rate': 0.0179845918984, 'max_cat_to_onehot': 11, 'min_data_in_leaf': 12, 'min_data_per_group': 3, 'num_leaves': 180}],
	'NN_TORCH': [{'activation': 'elu', 'ag_args': {'name_suffix': '_r37', 'priority': -4}, 'dropout_prob': 0.0889772897547275, 'hidden_size': 109, 'learning_rate': 0.02184363543226557, 'num_layers': 3, 'use_batchnorm': True, 'weight_decay': 3.1736637236578543e-10}, {'activation': 'elu', 'ag_args': {'name_suffix': '_r31', 'priority': -9}, 'dropout_prob': 0.013288954106470907, 'hidden_size': 81, 'learning_rate': 0.005340914647396153, 'num_layers': 4, 'use_batchnorm': False, 'weight_decay': 8.76216837077536e-05}, {'activation': 'elu', 'ag_args': {'name_suffix': '_r193', 'priority': -14}, 'dropout_prob': 0.2976404923811552, 'hidden_size': 131, 'learning_rate': 0.0038408014156739775, 'num_layers': 3, 'use_batchnorm': False, 'weight_decay': 0.01745189206113213}],
	'FASTAI': [{'ag_args': {'name_suffix': '_r25', 'priority': -6}, 'bs': 1024, 'emb_drop': 0.6167722379778131, 'epochs': 44, 'layers': [200, 100, 50], 'lr': 0.05344037785562929, 'ps': 0.48477211305443607}, {'ag_args': {'name_suffix': '_r162', 'priority': -11}, 'bs': 2048, 'emb_drop': 0.5474625640581479, 'epochs': 45, 'layers': [400, 200], 'lr': 0.0047438648957706655, 'ps': 0.07533239360470734}, {'ag_args': {'name_suffix': '_r147', 'priority': -20}, 'bs': 128, 'emb_drop': 0.6378380130337095, 'epochs': 48, 'layers': [200], 'lr': 0.058027179860229344, 'ps': 0.23253362133888375}],
}
User-specified callbacks (1): ['EarlyStoppingCountCallback']
EarlyStoppingCountCallback: Initializing patience to 18. Reason: num_rows_train=3448, patience_curve=[[100, 4], [500, 8], [2500, 15], [10000, 40], [100000, 100], None]
Fitting 50 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoost_c1_BAG_L1 ... Training model for up to 1161.50s of the 1161.49s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 63.76% memory usage per fold, 63.76%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=63.76%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	0.6491	 = Validation score   (accuracy)
	879.33s	 = Training   runtime
	2.89s	 = Validation runtime
Fitting model: LightGBMPrep_r13_BAG_L1 ... Training model for up to 274.62s of the 274.62s of remaining time.
	Warning: Potentially not enough memory to safely train model. Estimated to require 5.976 GB out of 7.139 GB available memory (83.705%)... (90.000% of avail memory is the max safe size)
	To avoid this warning, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.17 to avoid the warning)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 83.70% memory usage per fold, 83.70%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=83.70%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	Warning: Exception caused LightGBMPrep_r13_BAG_L1 to fail during training... Skipping this model.
		ray::_ray_fit() (pid=23089, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2201, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2085, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/stacker_ensemble_model.py", line 270, in _fit
    return super()._fit(X=X, y=y, time_limit=time_limit, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 393, in _fit
    self._fit_folds(
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 887, in _fit_folds
    fold_fitting_strategy.after_all_folds_scheduled()
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 794, in after_all_folds_scheduled
    self._run_pseudo_sequential(X, y, X_pseudo, y_pseudo, model_base_ref, time_limit_fold, head_node_id)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 752, in _run_pseudo_sequential
    self._process_fold_results(finished[0], unfinished, fold_ctx)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 666, in _process_fold_results
    raise processed_exception
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 619, in _process_fold_results
    out = self.ray.get(finished)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 2972, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 1031, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(LightGBMError): ray::_ray_fit() (pid=23089, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Fitting model: LightGBM_r177_BAG_L1 ... Training model for up to 263.71s of the 263.70s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 17.70% memory usage per fold, 70.79%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=17.70%)
	0.6459	 = Validation score   (accuracy)
	236.47s	 = Training   runtime
	1.22s	 = Validation runtime
Fitting model: NeuralNetTorch_r37_BAG_L1 ... Training model for up to 18.89s of the 18.89s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=2.44%)
	Time limit exceeded... Skipping NeuralNetTorch_r37_BAG_L1.
Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 3.74s of remaining time.
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=0.0/6.7 GB
	Ensemble Weights: {'CatBoost_c1_BAG_L1': 1.0}
	0.6491	 = Validation score   (accuracy)
	0.08s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 1196.42s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 149.1 rows/s (431 batch size)
Automatically performing refit_full as a post-fit operation (due to `.fit(..., refit_full=True)`
Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoost_c1_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=4.5/7.1 GB
	Many features detected (4557), dynamically setting 'colsample_bylevel' to 0.21944261575597981 to speed up training (Default = 1).
	To disable this functionality, explicitly specify 'colsample_bylevel' in the model hyperparameters.
	76.22s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r177_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=1.2/6.9 GB
	23.44s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'CatBoost_c1_BAG_L1': 1.0}
	0.08s	 = Training   runtime
Updated best model to "CatBoost_c1_BAG_L1_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "CatBoost_c1_BAG_L1_FULL" for predict() and predict_proba().
Refit complete, total runtime = 103.66s ... Best model: "CatBoost_c1_BAG_L1_FULL"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20260202_001125")
Loaded data from: evaluation.csv | Columns = 20 / 20 | Rows = 237 -> 237

```

## Ran with high_v150 and 110 Minutes, but completed earlier. Everything run on T4 GPU instance

```
 Collecting autogluon
  Downloading autogluon-1.5.0-py3-none-any.whl.metadata (12 kB)
Collecting autogluon.core==1.5.0 (from autogluon.core[all]==1.5.0->autogluon)
  Downloading autogluon_core-1.5.0-py3-none-any.whl.metadata (13 kB)
Collecting autogluon.features==1.5.0 (from autogluon)
  Downloading autogluon_features-1.5.0-py3-none-any.whl.metadata (12 kB)
Collecting autogluon.tabular==1.5.0 (from autogluon.tabular[all]==1.5.0->autogluon)
  Downloading autogluon_tabular-1.5.0-py3-none-any.whl.metadata (16 kB)
Collecting autogluon.multimodal==1.5.0 (from autogluon)
  Downloading autogluon_multimodal-1.5.0-py3-none-any.whl.metadata (13 kB)
Collecting autogluon.timeseries==1.5.0 (from autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading autogluon_timeseries-1.5.0-py3-none-any.whl.metadata (13 kB)
Requirement already satisfied: numpy<2.4.0,>=1.25.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.0.2)
Requirement already satisfied: scipy<1.17,>=1.5.4 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.16.3)
Requirement already satisfied: scikit-learn<1.8.0,>=1.4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.6.1)
Requirement already satisfied: networkx<4,>=3.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.6.1)
Requirement already satisfied: pandas<2.4.0,>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.2.2)
Requirement already satisfied: tqdm<5,>=4.38 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (4.67.1)
Requirement already satisfied: requests in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.32.4)
Requirement already satisfied: matplotlib<3.11,>=3.7.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.10.0)
Collecting boto3<2,>=1.10 (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon)
  Downloading boto3-1.42.39-py3-none-any.whl.metadata (6.8 kB)
Collecting autogluon.common==1.5.0 (from autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon)
  Downloading autogluon_common-1.5.0-py3-none-any.whl.metadata (12 kB)
Collecting ray<2.53,>=2.43.0 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading ray-2.52.1-cp312-cp312-manylinux2014_x86_64.whl.metadata (21 kB)
Requirement already satisfied: pyarrow>=15.0.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (18.1.0)
Requirement already satisfied: hyperopt<0.2.8,>=0.2.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.core[all]==1.5.0->autogluon) (0.2.7)
Collecting stevedore<5.5 (from autogluon.core[all]==1.5.0->autogluon)
  Downloading stevedore-5.4.1-py3-none-any.whl.metadata (2.3 kB)
Requirement already satisfied: Pillow<12,>=10.0.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (11.3.0)
Requirement already satisfied: torch<2.10,>=2.6 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.9.0+cu126)
Collecting lightning<2.6,>=2.5.1 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading lightning-2.5.6-py3-none-any.whl.metadata (42 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.2 MB/s eta 0:00:00
Requirement already satisfied: transformers<4.58,>=4.51.0 in /usr/local/lib/python3.12/dist-packages (from transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (4.57.6)
Requirement already satisfied: accelerate<2.0,>=0.34.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.12.0)
Requirement already satisfied: fsspec<=2025.3 in /usr/local/lib/python3.12/dist-packages (from fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (2025.3.0)
Collecting jsonschema<4.24,>=4.18 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
Collecting seqeval<1.3.0,>=1.2.2 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading seqeval-1.2.2.tar.gz (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Collecting evaluate<0.5.0,>=0.4.0 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
Collecting timm<1.0.7,>=0.9.5 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading timm-1.0.3-py3-none-any.whl.metadata (43 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
Requirement already satisfied: torchvision<0.25.0,>=0.21.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.24.0+cu126)
Requirement already satisfied: scikit-image<0.26.0,>=0.19.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.25.2)
Requirement already satisfied: text-unidecode<1.4,>=1.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (1.3)
Collecting torchmetrics<1.8,>=1.2.0 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading torchmetrics-1.7.4-py3-none-any.whl.metadata (21 kB)
Requirement already satisfied: omegaconf<2.4.0,>=2.1.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.3.0)
Collecting pytorch-metric-learning<2.9,>=1.3.0 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading pytorch_metric_learning-2.8.1-py3-none-any.whl.metadata (18 kB)
Collecting nlpaug<1.2.0,>=1.1.10 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading nlpaug-1.1.11-py3-none-any.whl.metadata (14 kB)
Requirement already satisfied: nltk<3.10,>=3.4.5 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (3.9.1)
Collecting openmim<0.4.0,>=0.3.7 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading openmim-0.3.9-py2.py3-none-any.whl.metadata (16 kB)
Requirement already satisfied: defusedxml<0.7.2,>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: jinja2<3.2,>=3.0.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (3.1.6)
Requirement already satisfied: tensorboard<3,>=2.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.multimodal==1.5.0->autogluon) (2.19.0)
Collecting pytesseract<0.4,>=0.3.9 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading pytesseract-0.3.13-py3-none-any.whl.metadata (11 kB)
Collecting nvidia-ml-py3<8.0,>=7.352.0 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading nvidia-ml-py3-7.352.0.tar.gz (19 kB)
  Preparing metadata (setup.py) ... done
Collecting pdf2image<1.19,>=1.17.0 (from autogluon.multimodal==1.5.0->autogluon)
  Downloading pdf2image-1.17.0-py3-none-any.whl.metadata (6.2 kB)
Collecting einx (from autogluon.tabular[all]==1.5.0->autogluon)
  Downloading einx-0.3.0-py3-none-any.whl.metadata (6.9 kB)
Requirement already satisfied: lightgbm<4.7,>=4.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (4.6.0)
Requirement already satisfied: huggingface_hub<1.0 in /usr/local/lib/python3.12/dist-packages (from huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (0.36.0)
Requirement already satisfied: xgboost<3.2,>=2.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (3.1.3)
Collecting catboost<1.3,>=1.2 (from autogluon.tabular[all]==1.5.0->autogluon)
  Downloading catboost-1.2.8-cp312-cp312-manylinux2014_x86_64.whl.metadata (1.2 kB)
Requirement already satisfied: fastai<2.9,>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (2.8.6)
Collecting loguru (from autogluon.tabular[all]==1.5.0->autogluon)
  Downloading loguru-0.7.3-py3-none-any.whl.metadata (22 kB)
Requirement already satisfied: einops<0.9,>=0.7 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (0.8.2)
Requirement already satisfied: spacy<3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.tabular[all]==1.5.0->autogluon) (3.8.11)
Requirement already satisfied: joblib<1.7,>=1.2 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.5.3)
Collecting gluonts<0.17,>=0.15.0 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading gluonts-0.16.2-py3-none-any.whl.metadata (9.8 kB)
Collecting statsforecast<2.0.2,>=1.7.0 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading statsforecast-2.0.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (29 kB)
Collecting mlforecast<0.15.0,>=0.14.0 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading mlforecast-0.14.0-py3-none-any.whl.metadata (12 kB)
Collecting utilsforecast<0.2.12,>=0.2.3 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading utilsforecast-0.2.11-py3-none-any.whl.metadata (7.7 kB)
Collecting coreforecast<0.0.17,>=0.0.12 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading coreforecast-0.0.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.7 kB)
Collecting fugue>=0.9.0 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading fugue-0.9.6-py3-none-any.whl.metadata (18 kB)
Requirement already satisfied: orjson~=3.9 in /usr/local/lib/python3.12/dist-packages (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (3.11.5)
Collecting chronos-forecasting<2.4,>=2.2.2 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading chronos_forecasting-2.2.2-py3-none-any.whl.metadata (23 kB)
Collecting peft<0.18,>=0.13.0 (from autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading peft-0.17.1-py3-none-any.whl.metadata (14 kB)
Requirement already satisfied: psutil<7.2.0,>=5.7.3 in /usr/local/lib/python3.12/dist-packages (from autogluon.common==1.5.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (5.9.5)
Requirement already satisfied: pyyaml>=5.0 in /usr/local/lib/python3.12/dist-packages (from autogluon.common==1.5.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (6.0.3)
Requirement already satisfied: packaging>=20.0 in /usr/local/lib/python3.12/dist-packages (from accelerate<2.0,>=0.34.0->autogluon.multimodal==1.5.0->autogluon) (25.0)
Requirement already satisfied: safetensors>=0.4.3 in /usr/local/lib/python3.12/dist-packages (from accelerate<2.0,>=0.34.0->autogluon.multimodal==1.5.0->autogluon) (0.7.0)
Collecting botocore<1.43.0,>=1.42.39 (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon)
  Downloading botocore-1.42.39-py3-none-any.whl.metadata (5.9 kB)
Collecting jmespath<2.0.0,>=0.7.1 (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon)
  Downloading jmespath-1.1.0-py3-none-any.whl.metadata (7.6 kB)
Collecting s3transfer<0.17.0,>=0.16.0 (from boto3<2,>=1.10->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon)
  Downloading s3transfer-0.16.0-py3-none-any.whl.metadata (1.7 kB)
Requirement already satisfied: graphviz in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (0.21)
Requirement already satisfied: plotly in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (5.24.1)
Requirement already satisfied: six in /usr/local/lib/python3.12/dist-packages (from catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (1.17.0)
Requirement already satisfied: datasets>=2.0.0 in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (4.0.0)
Requirement already satisfied: dill in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (0.3.8)
Requirement already satisfied: xxhash in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (3.6.0)
Requirement already satisfied: multiprocess in /usr/local/lib/python3.12/dist-packages (from evaluate<0.5.0,>=0.4.0->autogluon.multimodal==1.5.0->autogluon) (0.70.16)
Requirement already satisfied: pip in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (24.1.2)
Requirement already satisfied: fastdownload<2,>=0.0.5 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.7)
Requirement already satisfied: fastcore>=1.8.0 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.12.7)
Requirement already satisfied: fasttransform>=0.0.2 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.2)
Requirement already satisfied: fastprogress>=0.2.4 in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.1.3)
Requirement already satisfied: plum-dispatch in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: cloudpickle in /usr/local/lib/python3.12/dist-packages (from fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.1.2)
Requirement already satisfied: aiohttp!=4.0.0a0,!=4.0.0a1 in /usr/local/lib/python3.12/dist-packages (from fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (3.13.3)
Collecting triad>=1.0.0 (from fugue>=0.9.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading triad-1.0.0-py3-none-any.whl.metadata (6.6 kB)
Collecting adagio>=0.2.6 (from fugue>=0.9.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading adagio-0.2.6-py3-none-any.whl.metadata (1.8 kB)
Requirement already satisfied: pydantic<3,>=1.7 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.12.3)
Requirement already satisfied: toolz~=0.10 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.12.1)
Requirement already satisfied: typing-extensions~=4.0 in /usr/local/lib/python3.12/dist-packages (from gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (4.15.0)
Requirement already satisfied: filelock in /usr/local/lib/python3.12/dist-packages (from huggingface_hub<1.0->huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (3.20.3)
Requirement already satisfied: hf-xet<2.0.0,>=1.1.3 in /usr/local/lib/python3.12/dist-packages (from huggingface_hub<1.0->huggingface_hub[torch]<1.0; extra == "all"->autogluon.tabular[all]==1.5.0->autogluon) (1.2.0)
Requirement already satisfied: future in /usr/local/lib/python3.12/dist-packages (from hyperopt<0.2.8,>=0.2.7->autogluon.core[all]==1.5.0->autogluon) (1.0.0)
Requirement already satisfied: py4j in /usr/local/lib/python3.12/dist-packages (from hyperopt<0.2.8,>=0.2.7->autogluon.core[all]==1.5.0->autogluon) (0.10.9.9)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.12/dist-packages (from jinja2<3.2,>=3.0.3->autogluon.multimodal==1.5.0->autogluon) (3.0.3)
Requirement already satisfied: attrs>=22.2.0 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (25.4.0)
Requirement already satisfied: jsonschema-specifications>=2023.03.6 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (2025.9.1)
Requirement already satisfied: referencing>=0.28.4 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (0.37.0)
Requirement already satisfied: rpds-py>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from jsonschema<4.24,>=4.18->autogluon.multimodal==1.5.0->autogluon) (0.30.0)
Collecting lightning-utilities<2.0,>=0.10.0 (from lightning<2.6,>=2.5.1->autogluon.multimodal==1.5.0->autogluon)
  Downloading lightning_utilities-0.15.2-py3-none-any.whl.metadata (5.7 kB)
Collecting pytorch-lightning (from lightning<2.6,>=2.5.1->autogluon.multimodal==1.5.0->autogluon)
  Downloading pytorch_lightning-2.6.1-py3-none-any.whl.metadata (21 kB)
Requirement already satisfied: contourpy>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.3.3)
Requirement already satisfied: cycler>=0.10 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (0.12.1)
Requirement already satisfied: fonttools>=4.22.0 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (4.61.1)
Requirement already satisfied: kiwisolver>=1.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (1.4.9)
Requirement already satisfied: pyparsing>=2.3.1 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.3.2)
Requirement already satisfied: python-dateutil>=2.7 in /usr/local/lib/python3.12/dist-packages (from matplotlib<3.11,>=3.7.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.9.0.post0)
Requirement already satisfied: numba in /usr/local/lib/python3.12/dist-packages (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.60.0)
Collecting optuna (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading optuna-4.7.0-py3-none-any.whl.metadata (17 kB)
Collecting window-ops (from mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading window_ops-0.0.15-py3-none-any.whl.metadata (6.8 kB)
Requirement already satisfied: gdown>=4.0.0 in /usr/local/lib/python3.12/dist-packages (from nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (5.2.1)
Requirement already satisfied: click in /usr/local/lib/python3.12/dist-packages (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon) (8.3.1)
Requirement already satisfied: regex>=2021.8.3 in /usr/local/lib/python3.12/dist-packages (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon) (2025.11.3)
Requirement already satisfied: antlr4-python3-runtime==4.9.* in /usr/local/lib/python3.12/dist-packages (from omegaconf<2.4.0,>=2.1.1->autogluon.multimodal==1.5.0->autogluon) (4.9.3)
Collecting colorama (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading colorama-0.4.6-py2.py3-none-any.whl.metadata (17 kB)
Collecting model-index (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading model_index-0.1.11-py3-none-any.whl.metadata (3.9 kB)
Collecting opendatalab (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading opendatalab-0.0.10-py3-none-any.whl.metadata (6.4 kB)
Requirement already satisfied: rich in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (13.9.4)
Requirement already satisfied: tabulate in /usr/local/lib/python3.12/dist-packages (from openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.9.0)
Requirement already satisfied: pytz>=2020.1 in /usr/local/lib/python3.12/dist-packages (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2025.2)
Requirement already satisfied: tzdata>=2022.7 in /usr/local/lib/python3.12/dist-packages (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2025.3)
Collecting click (from nltk<3.10,>=3.4.5->autogluon.multimodal==1.5.0->autogluon)
  Downloading click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
Requirement already satisfied: msgpack<2.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from ray<2.53,>=2.43.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.1.2)
Requirement already satisfied: protobuf>=3.20.3 in /usr/local/lib/python3.12/dist-packages (from ray<2.53,>=2.43.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (5.29.5)
Collecting aiohttp_cors (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading aiohttp_cors-0.8.1-py3-none-any.whl.metadata (20 kB)
Collecting colorful (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading colorful-0.5.8-py2.py3-none-any.whl.metadata (17 kB)
Collecting py-spy>=0.4.0 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading py_spy-0.4.1-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl.metadata (510 bytes)
Requirement already satisfied: grpcio>=1.42.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.76.0)
Collecting opencensus (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opencensus-0.11.4-py2.py3-none-any.whl.metadata (12 kB)
Requirement already satisfied: opentelemetry-sdk>=1.30.0 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Collecting opentelemetry-exporter-prometheus (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_exporter_prometheus-0.60b1-py3-none-any.whl.metadata (2.1 kB)
Requirement already satisfied: opentelemetry-proto in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: prometheus_client>=0.7.1 in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.24.1)
Requirement already satisfied: smart_open in /usr/local/lib/python3.12/dist-packages (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (7.5.0)
Collecting virtualenv!=20.21.1,>=20.0.24 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading virtualenv-20.36.1-py3-none-any.whl.metadata (4.7 kB)
Collecting tensorboardX>=1.9 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading tensorboardx-2.6.4-py3-none-any.whl.metadata (6.2 kB)
Requirement already satisfied: charset_normalizer<4,>=2 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.4.4)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.11)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2.5.0)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.12/dist-packages (from requests->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (2026.1.4)
Requirement already satisfied: imageio!=2.35.0,>=2.33 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (2.37.2)
Requirement already satisfied: tifffile>=2022.8.12 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (2026.1.14)
Requirement already satisfied: lazy-loader>=0.4 in /usr/local/lib/python3.12/dist-packages (from scikit-image<0.26.0,>=0.19.1->autogluon.multimodal==1.5.0->autogluon) (0.4)
Requirement already satisfied: threadpoolctl>=3.1.0 in /usr/local/lib/python3.12/dist-packages (from scikit-learn<1.8.0,>=1.4.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon) (3.6.0)
Requirement already satisfied: spacy-legacy<3.1.0,>=3.0.11 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (3.0.12)
Requirement already satisfied: spacy-loggers<2.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.0.5)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.0.15)
Requirement already satisfied: cymem<2.1.0,>=2.0.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.0.13)
Requirement already satisfied: preshed<3.1.0,>=3.0.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (3.0.12)
Requirement already satisfied: thinc<8.4.0,>=8.3.4 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (8.3.10)
Requirement already satisfied: wasabi<1.2.0,>=0.9.1 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.1.3)
Requirement already satisfied: srsly<3.0.0,>=2.4.3 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.5.2)
Requirement already satisfied: catalogue<2.1.0,>=2.0.6 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (2.0.10)
Requirement already satisfied: weasel<0.5.0,>=0.4.2 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.4.3)
Requirement already satisfied: typer-slim<1.0.0,>=0.3.0 in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.21.1)
Requirement already satisfied: setuptools in /usr/local/lib/python3.12/dist-packages (from spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (75.2.0)
Requirement already satisfied: statsmodels>=0.13.2 in /usr/local/lib/python3.12/dist-packages (from statsforecast<2.0.2,>=1.7.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.14.6)
Collecting pbr>=2.0.0 (from stevedore<5.5->autogluon.core[all]==1.5.0->autogluon)
  Downloading pbr-7.0.3-py2.py3-none-any.whl.metadata (3.8 kB)
Requirement already satisfied: absl-py>=0.4 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (1.4.0)
Requirement already satisfied: markdown>=2.6.8 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (3.10.1)
Requirement already satisfied: tensorboard-data-server<0.8.0,>=0.7.0 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (0.7.2)
Requirement already satisfied: werkzeug>=1.0.1 in /usr/local/lib/python3.12/dist-packages (from tensorboard<3,>=2.9->autogluon.multimodal==1.5.0->autogluon) (3.1.5)
Requirement already satisfied: sympy>=1.13.3 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.14.0)
Requirement already satisfied: nvidia-cuda-nvrtc-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-cuda-runtime-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-cuda-cupti-cu12==12.6.80 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.80)
Requirement already satisfied: nvidia-cudnn-cu12==9.10.2.21 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (9.10.2.21)
Requirement already satisfied: nvidia-cublas-cu12==12.6.4.1 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.4.1)
Requirement already satisfied: nvidia-cufft-cu12==11.3.0.4 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (11.3.0.4)
Requirement already satisfied: nvidia-curand-cu12==10.3.7.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (10.3.7.77)
Requirement already satisfied: nvidia-cusolver-cu12==11.7.1.2 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (11.7.1.2)
Requirement already satisfied: nvidia-cusparse-cu12==12.5.4.2 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.5.4.2)
Requirement already satisfied: nvidia-cusparselt-cu12==0.7.1 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: nvidia-nccl-cu12==2.27.5 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (2.27.5)
Requirement already satisfied: nvidia-nvshmem-cu12==3.3.20 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (3.3.20)
Requirement already satisfied: nvidia-nvtx-cu12==12.6.77 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.77)
Requirement already satisfied: nvidia-nvjitlink-cu12==12.6.85 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (12.6.85)
Requirement already satisfied: nvidia-cufile-cu12==1.11.1.6 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.11.1.6)
Requirement already satisfied: triton==3.5.0 in /usr/local/lib/python3.12/dist-packages (from torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (3.5.0)
Requirement already satisfied: tokenizers<=0.23.0,>=0.22.0 in /usr/local/lib/python3.12/dist-packages (from transformers<4.58,>=4.51.0->transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (0.22.2)
Requirement already satisfied: sentencepiece!=0.1.92,>=0.1.91 in /usr/local/lib/python3.12/dist-packages (from transformers[sentencepiece]<4.58,>=4.51.0->autogluon.multimodal==1.5.0->autogluon) (0.2.1)
Requirement already satisfied: frozendict in /usr/local/lib/python3.12/dist-packages (from einx->autogluon.tabular[all]==1.5.0->autogluon) (2.4.7)
Requirement already satisfied: aiohappyeyeballs>=2.5.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (2.6.1)
Requirement already satisfied: aiosignal>=1.4.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.4.0)
Requirement already satisfied: frozenlist>=1.1.1 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.8.0)
Requirement already satisfied: multidict<7.0,>=4.5 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (6.7.1)
Requirement already satisfied: propcache>=0.2.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (0.4.1)
Requirement already satisfied: yarl<2.0,>=1.17.0 in /usr/local/lib/python3.12/dist-packages (from aiohttp!=4.0.0a0,!=4.0.0a1->fsspec[http]<=2025.3->autogluon.multimodal==1.5.0->autogluon) (1.22.0)
Requirement already satisfied: python-fasthtml>=0.12.34 in /usr/local/lib/python3.12/dist-packages (from fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.12.39)
Requirement already satisfied: beautifulsoup4 in /usr/local/lib/python3.12/dist-packages (from gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (4.13.5)
Requirement already satisfied: llvmlite<0.44,>=0.43.0dev0 in /usr/local/lib/python3.12/dist-packages (from numba->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.43.0)
Requirement already satisfied: opentelemetry-api==1.37.0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.37.0)
Requirement already satisfied: opentelemetry-semantic-conventions==0.58b0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.58b0)
Requirement already satisfied: importlib-metadata<8.8.0,>=6.0 in /usr/local/lib/python3.12/dist-packages (from opentelemetry-api==1.37.0->opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (8.7.1)
Requirement already satisfied: annotated-types>=0.6.0 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.7.0)
Requirement already satisfied: pydantic-core==2.41.4 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.41.4)
Requirement already satisfied: typing-inspection>=0.4.2 in /usr/local/lib/python3.12/dist-packages (from pydantic<3,>=1.7->gluonts<0.17,>=0.15.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (0.4.2)
Requirement already satisfied: patsy>=0.5.6 in /usr/local/lib/python3.12/dist-packages (from statsmodels>=0.13.2->statsforecast<2.0.2,>=1.7.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.0.2)
Requirement already satisfied: mpmath<1.4,>=1.1.0 in /usr/local/lib/python3.12/dist-packages (from sympy>=1.13.3->torch<2.10,>=2.6->autogluon.multimodal==1.5.0->autogluon) (1.3.0)
Requirement already satisfied: blis<1.4.0,>=1.3.0 in /usr/local/lib/python3.12/dist-packages (from thinc<8.4.0,>=8.3.4->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (1.3.3)
Requirement already satisfied: confection<1.0.0,>=0.0.1 in /usr/local/lib/python3.12/dist-packages (from thinc<8.4.0,>=8.3.4->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.1.5)
Collecting distlib<1,>=0.3.7 (from virtualenv!=20.21.1,>=20.0.24->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading distlib-0.4.0-py2.py3-none-any.whl.metadata (5.2 kB)
Requirement already satisfied: platformdirs<5,>=3.9.1 in /usr/local/lib/python3.12/dist-packages (from virtualenv!=20.21.1,>=20.0.24->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (4.5.1)
Requirement already satisfied: cloudpathlib<1.0.0,>=0.7.0 in /usr/local/lib/python3.12/dist-packages (from weasel<0.5.0,>=0.4.2->spacy<3.9->autogluon.tabular[all]==1.5.0->autogluon) (0.23.0)
Requirement already satisfied: wrapt in /usr/local/lib/python3.12/dist-packages (from smart_open->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.0.1)
Collecting ordered-set (from model-index->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading ordered_set-4.1.0-py3-none-any.whl.metadata (5.3 kB)
Collecting opencensus-context>=0.1.3 (from opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opencensus_context-0.1.3-py2.py3-none-any.whl.metadata (3.3 kB)
Requirement already satisfied: google-api-core<3.0.0,>=1.0.0 in /usr/local/lib/python3.12/dist-packages (from opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.29.0)
Collecting pycryptodome (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading pycryptodome-3.23.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
Collecting openxlab (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading openxlab-0.1.3-py3-none-any.whl.metadata (3.8 kB)
Collecting opentelemetry-sdk>=1.30.0 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_sdk-1.39.1-py3-none-any.whl.metadata (1.5 kB)
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
Collecting opentelemetry-exporter-prometheus (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_exporter_prometheus-0.60b0-py3-none-any.whl.metadata (2.1 kB)
Collecting opentelemetry-sdk>=1.30.0 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_sdk-1.39.0-py3-none-any.whl.metadata (1.5 kB)
Collecting opentelemetry-exporter-prometheus (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_exporter_prometheus-0.59b0-py3-none-any.whl.metadata (2.1 kB)
Collecting opentelemetry-sdk>=1.30.0 (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
Collecting opentelemetry-exporter-prometheus (from ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon)
  Downloading opentelemetry_exporter_prometheus-0.58b0-py3-none-any.whl.metadata (2.1 kB)
Requirement already satisfied: alembic>=1.5.0 in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.18.1)
Collecting colorlog (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon)
  Downloading colorlog-6.10.1-py3-none-any.whl.metadata (11 kB)
Requirement already satisfied: sqlalchemy>=1.4.2 in /usr/local/lib/python3.12/dist-packages (from optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (2.0.46)
Requirement already satisfied: tenacity>=6.2.0 in /usr/local/lib/python3.12/dist-packages (from plotly->catboost<1.3,>=1.2->autogluon.tabular[all]==1.5.0->autogluon) (9.1.2)
Requirement already satisfied: beartype>=0.16.2 in /usr/local/lib/python3.12/dist-packages (from plum-dispatch->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.22.9)
Requirement already satisfied: markdown-it-py>=2.2.0 in /usr/local/lib/python3.12/dist-packages (from rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (4.0.0)
Requirement already satisfied: pygments<3.0.0,>=2.13.0 in /usr/local/lib/python3.12/dist-packages (from rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (2.19.2)
Requirement already satisfied: Mako in /usr/local/lib/python3.12/dist-packages (from alembic>=1.5.0->optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (1.3.10)
Requirement already satisfied: googleapis-common-protos<2.0.0,>=1.56.2 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.72.0)
Requirement already satisfied: proto-plus<2.0.0,>=1.22.3 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (1.27.0)
Requirement already satisfied: google-auth<3.0.0,>=2.14.1 in /usr/local/lib/python3.12/dist-packages (from google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (2.47.0)
Requirement already satisfied: mdurl~=0.1 in /usr/local/lib/python3.12/dist-packages (from markdown-it-py>=2.2.0->rich->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon) (0.1.2)
Requirement already satisfied: starlette>0.33 in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.50.0)
Requirement already satisfied: oauthlib in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.3.1)
Requirement already satisfied: itsdangerous in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (2.2.0)
Requirement already satisfied: uvicorn>=0.30 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.40.0)
Requirement already satisfied: httpx in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.28.1)
Requirement already satisfied: fastlite>=0.1.1 in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.2.4)
Requirement already satisfied: python-multipart in /usr/local/lib/python3.12/dist-packages (from python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.0.22)
Requirement already satisfied: greenlet>=1 in /usr/local/lib/python3.12/dist-packages (from sqlalchemy>=1.4.2->optuna->mlforecast<0.15.0,>=0.14.0->autogluon.timeseries==1.5.0->autogluon.timeseries[all]==1.5.0->autogluon) (3.3.1)
Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.12/dist-packages (from beautifulsoup4->gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (2.8.3)
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
Collecting openxlab (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading openxlab-0.1.2-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.1.1-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.1.0-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.38-py3-none-any.whl.metadata (3.8 kB)
Collecting oss2~=2.17.0 (from openxlab->opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading oss2-2.17.0.tar.gz (259 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Collecting pytz>=2020.1 (from pandas<2.4.0,>=2.0.0->autogluon.core==1.5.0->autogluon.core[all]==1.5.0->autogluon)
  Downloading pytz-2023.4-py2.py3-none-any.whl.metadata (22 kB)
Collecting openxlab (from opendatalab->openmim<0.4.0,>=0.3.7->autogluon.multimodal==1.5.0->autogluon)
  Downloading openxlab-0.0.37-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.36-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.35-py3-none-any.whl.metadata (3.8 kB)
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
  Downloading openxlab-0.0.34-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.33-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.32-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.31-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.30-py3-none-any.whl.metadata (3.8 kB)
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
  Downloading openxlab-0.0.29-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.28-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.27-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.26-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.25-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.24-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.23-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.22-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.21-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.20-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.19-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.18-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.17-py3-none-any.whl.metadata (3.7 kB)
  Downloading openxlab-0.0.16-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.15-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.14-py3-none-any.whl.metadata (3.8 kB)
  Downloading openxlab-0.0.13-py3-none-any.whl.metadata (4.5 kB)
  Downloading openxlab-0.0.12-py3-none-any.whl.metadata (4.5 kB)
  Downloading openxlab-0.0.11-py3-none-any.whl.metadata (4.3 kB)
Requirement already satisfied: PySocks!=1.5.7,>=1.5.6 in /usr/local/lib/python3.12/dist-packages (from requests[socks]->gdown>=4.0.0->nlpaug<1.2.0,>=1.1.10->autogluon.multimodal==1.5.0->autogluon) (1.7.1)
Requirement already satisfied: apswutils>=0.1.2 in /usr/local/lib/python3.12/dist-packages (from fastlite>=0.1.1->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.1.2)
Requirement already satisfied: pyasn1-modules>=0.2.1 in /usr/local/lib/python3.12/dist-packages (from google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.4.2)
Requirement already satisfied: rsa<5,>=3.1.4 in /usr/local/lib/python3.12/dist-packages (from google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (4.9.1)
Requirement already satisfied: zipp>=3.20 in /usr/local/lib/python3.12/dist-packages (from importlib-metadata<8.8.0,>=6.0->opentelemetry-api==1.37.0->opentelemetry-sdk>=1.30.0->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (3.23.0)
Requirement already satisfied: anyio<5,>=3.6.2 in /usr/local/lib/python3.12/dist-packages (from starlette>0.33->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (4.12.1)
Requirement already satisfied: h11>=0.8 in /usr/local/lib/python3.12/dist-packages (from uvicorn>=0.30->uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.16.0)
Requirement already satisfied: httptools>=0.6.3 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.7.1)
Requirement already satisfied: python-dotenv>=0.13 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.2.1)
Requirement already satisfied: uvloop>=0.15.1 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (0.22.1)
Requirement already satisfied: watchfiles>=0.13 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.1.1)
Requirement already satisfied: websockets>=10.4 in /usr/local/lib/python3.12/dist-packages (from uvicorn[standard]>=0.30->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (15.0.1)
Requirement already satisfied: httpcore==1.* in /usr/local/lib/python3.12/dist-packages (from httpx->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (1.0.9)
Requirement already satisfied: apsw in /usr/local/lib/python3.12/dist-packages (from apswutils>=0.1.2->fastlite>=0.1.1->python-fasthtml>=0.12.34->fastprogress>=0.2.4->fastai<2.9,>=2.3.1->autogluon.tabular[all]==1.5.0->autogluon) (3.51.2.0)
Requirement already satisfied: pyasn1<0.7.0,>=0.6.1 in /usr/local/lib/python3.12/dist-packages (from pyasn1-modules>=0.2.1->google-auth<3.0.0,>=2.14.1->google-api-core<3.0.0,>=1.0.0->opencensus->ray[default,tune]<2.53,>=2.43.0; (platform_system != "Windows" or python_version != "3.13") and extra == "all"->autogluon.core[all]==1.5.0->autogluon) (0.6.2)
Downloading autogluon-1.5.0-py3-none-any.whl (5.9 kB)
Downloading autogluon_core-1.5.0-py3-none-any.whl (227 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.1 MB/s eta 0:00:00
Downloading autogluon_features-1.5.0-py3-none-any.whl (98 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 7.4 MB/s eta 0:00:00
Downloading autogluon_multimodal-1.5.0-py3-none-any.whl (452 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.1/452.1 kB 25.4 MB/s eta 0:00:00
Downloading autogluon_tabular-1.5.0-py3-none-any.whl (515 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 32.5 MB/s eta 0:00:00
Downloading autogluon_timeseries-1.5.0-py3-none-any.whl (244 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 15.7 MB/s eta 0:00:00
Downloading autogluon_common-1.5.0-py3-none-any.whl (74 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 7.8 MB/s eta 0:00:00
Downloading boto3-1.42.39-py3-none-any.whl (140 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 10.8 MB/s eta 0:00:00
Downloading catboost-1.2.8-cp312-cp312-manylinux2014_x86_64.whl (99.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.4 MB/s eta 0:00:00
Downloading chronos_forecasting-2.2.2-py3-none-any.whl (72 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 kB 7.3 MB/s eta 0:00:00
Downloading coreforecast-0.0.16-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (287 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 28.4 MB/s eta 0:00:00
Downloading evaluate-0.4.6-py3-none-any.whl (84 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
Downloading fugue-0.9.6-py3-none-any.whl (280 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 kB 26.5 MB/s eta 0:00:00
Downloading gluonts-0.16.2-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 60.0 MB/s eta 0:00:00
Downloading jsonschema-4.23.0-py3-none-any.whl (88 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.5/88.5 kB 10.0 MB/s eta 0:00:00
Downloading lightning-2.5.6-py3-none-any.whl (827 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.9/827.9 kB 62.5 MB/s eta 0:00:00
Downloading mlforecast-0.14.0-py3-none-any.whl (71 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
Downloading nlpaug-1.1.11-py3-none-any.whl (410 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 37.5 MB/s eta 0:00:00
Downloading openmim-0.3.9-py2.py3-none-any.whl (52 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 6.2 MB/s eta 0:00:00
Downloading pdf2image-1.17.0-py3-none-any.whl (11 kB)
Downloading peft-0.17.1-py3-none-any.whl (504 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.9/504.9 kB 45.8 MB/s eta 0:00:00
Downloading pytesseract-0.3.13-py3-none-any.whl (14 kB)
Downloading pytorch_metric_learning-2.8.1-py3-none-any.whl (125 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 14.1 MB/s eta 0:00:00
Downloading ray-2.52.1-cp312-cp312-manylinux2014_x86_64.whl (72.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 11.9 MB/s eta 0:00:00
Downloading statsforecast-2.0.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (353 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.3/353.3 kB 33.0 MB/s eta 0:00:00
Downloading stevedore-5.4.1-py3-none-any.whl (49 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
Downloading timm-1.0.3-py3-none-any.whl (2.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 102.1 MB/s eta 0:00:00
Downloading torchmetrics-1.7.4-py3-none-any.whl (963 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 72.2 MB/s eta 0:00:00
Downloading utilsforecast-0.2.11-py3-none-any.whl (41 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
Downloading einx-0.3.0-py3-none-any.whl (102 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.0/103.0 kB 10.2 MB/s eta 0:00:00
Downloading loguru-0.7.3-py3-none-any.whl (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.3 MB/s eta 0:00:00
Downloading adagio-0.2.6-py3-none-any.whl (19 kB)
Downloading botocore-1.42.39-py3-none-any.whl (14.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 143.5 MB/s eta 0:00:00
Downloading click-8.2.1-py3-none-any.whl (102 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 12.5 MB/s eta 0:00:00
Downloading jmespath-1.1.0-py3-none-any.whl (20 kB)
Downloading lightning_utilities-0.15.2-py3-none-any.whl (29 kB)
Downloading pbr-7.0.3-py2.py3-none-any.whl (131 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.9/131.9 kB 14.7 MB/s eta 0:00:00
Downloading py_spy-0.4.1-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (2.8 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 114.3 MB/s eta 0:00:00
Downloading s3transfer-0.16.0-py3-none-any.whl (86 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00
Downloading tensorboardx-2.6.4-py3-none-any.whl (87 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.3 MB/s eta 0:00:00
Downloading triad-1.0.0-py3-none-any.whl (59 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.6 MB/s eta 0:00:00
Downloading virtualenv-20.36.1-py3-none-any.whl (6.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 135.7 MB/s eta 0:00:00
Downloading aiohttp_cors-0.8.1-py3-none-any.whl (25 kB)
Downloading colorama-0.4.6-py2.py3-none-any.whl (25 kB)
Downloading colorful-0.5.8-py2.py3-none-any.whl (201 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 kB 19.5 MB/s eta 0:00:00
Downloading model_index-0.1.11-py3-none-any.whl (34 kB)
Downloading opencensus-0.11.4-py2.py3-none-any.whl (128 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 14.3 MB/s eta 0:00:00
Downloading opendatalab-0.0.10-py3-none-any.whl (29 kB)
Downloading opentelemetry_exporter_prometheus-0.58b0-py3-none-any.whl (13 kB)
Downloading optuna-4.7.0-py3-none-any.whl (413 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 37.0 MB/s eta 0:00:00
Downloading pytorch_lightning-2.6.1-py3-none-any.whl (857 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.3/857.3 kB 68.4 MB/s eta 0:00:00
Downloading window_ops-0.0.15-py3-none-any.whl (15 kB)
Downloading distlib-0.4.0-py2.py3-none-any.whl (469 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 43.5 MB/s eta 0:00:00
Downloading opencensus_context-0.1.3-py2.py3-none-any.whl (5.1 kB)
Downloading colorlog-6.10.1-py3-none-any.whl (11 kB)
Downloading openxlab-0.0.11-py3-none-any.whl (55 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 5.7 MB/s eta 0:00:00
Downloading ordered_set-4.1.0-py3-none-any.whl (7.6 kB)
Downloading pycryptodome-3.23.0-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 107.5 MB/s eta 0:00:00
Building wheels for collected packages: nvidia-ml-py3, seqeval
  Building wheel for nvidia-ml-py3 (setup.py) ... done
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19172 sha256=ec5f67da83cffd40b1bd92937ec8272a2c7dd10ba0018782994550b613b1c19a
  Stored in directory: /root/.cache/pip/wheels/6e/65/79/33dee66cba26e8204801916dfee7481bccfd22905ebb841fe5
  Building wheel for seqeval (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6d94095b2ed69fc8dce781e332f980ba1f52dfad70ebc79c391e78731f356000
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built nvidia-ml-py3 seqeval
Installing collected packages: py-spy, opencensus-context, nvidia-ml-py3, distlib, colorful, virtualenv, tensorboardX, pytesseract, pycryptodome, pdf2image, pbr, ordered-set, openxlab, loguru, lightning-utilities, jmespath, coreforecast, colorlog, colorama, click, window-ops, stevedore, model-index, einx, botocore, utilsforecast, triad, seqeval, s3transfer, optuna, opendatalab, jsonschema, gluonts, catboost, aiohttp_cors, torchmetrics, ray, pytorch-metric-learning, openmim, opencensus, nlpaug, mlforecast, boto3, adagio, timm, pytorch-lightning, peft, opentelemetry-exporter-prometheus, fugue, evaluate, chronos-forecasting, autogluon.common, statsforecast, lightning, autogluon.features, autogluon.core, autogluon.tabular, autogluon.multimodal, autogluon.timeseries, autogluon
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.26.0
    Uninstalling jsonschema-4.26.0:
      Successfully uninstalled jsonschema-4.26.0
  Attempting uninstall: timm
    Found existing installation: timm 1.0.24
    Uninstalling timm-1.0.24:
      Successfully uninstalled timm-1.0.24
  Attempting uninstall: peft
    Found existing installation: peft 0.18.1
    Uninstalling peft-0.18.1:
      Successfully uninstalled peft-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasterio 1.5.0 requires click!=8.2.*,>=4.0, but you have click 8.2.1 which is incompatible.
Successfully installed adagio-0.2.6 aiohttp_cors-0.8.1 autogluon-1.5.0 autogluon.common-1.5.0 autogluon.core-1.5.0 autogluon.features-1.5.0 autogluon.multimodal-1.5.0 autogluon.tabular-1.5.0 autogluon.timeseries-1.5.0 boto3-1.42.39 botocore-1.42.39 catboost-1.2.8 chronos-forecasting-2.2.2 click-8.2.1 colorama-0.4.6 colorful-0.5.8 colorlog-6.10.1 coreforecast-0.0.16 distlib-0.4.0 einx-0.3.0 evaluate-0.4.6 fugue-0.9.6 gluonts-0.16.2 jmespath-1.1.0 jsonschema-4.23.0 lightning-2.5.6 lightning-utilities-0.15.2 loguru-0.7.3 mlforecast-0.14.0 model-index-0.1.11 nlpaug-1.1.11 nvidia-ml-py3-7.352.0 opencensus-0.11.4 opencensus-context-0.1.3 opendatalab-0.0.10 openmim-0.3.9 opentelemetry-exporter-prometheus-0.58b0 openxlab-0.0.11 optuna-4.7.0 ordered-set-4.1.0 pbr-7.0.3 pdf2image-1.17.0 peft-0.17.1 py-spy-0.4.1 pycryptodome-3.23.0 pytesseract-0.3.13 pytorch-lightning-2.6.1 pytorch-metric-learning-2.8.1 ray-2.52.1 s3transfer-0.16.0 seqeval-1.2.2 statsforecast-2.0.1 stevedore-5.4.1 tensorboardX-2.6.4 timm-1.0.3 torchmetrics-1.7.4 triad-1.0.0 utilsforecast-0.2.11 virtualenv-20.36.1 window-ops-0.0.15
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
README.md:  12.7k/? [00:00<00:00, 934kB/s]data/train-00000-of-00001.parquet: 100% 3.90M/3.90M [00:00<00:00, 6.90MB/s]data/test-00000-of-00001.parquet: 100% 259k/259k [00:00<00:00, 1.09MB/s]Generating train split: 100% 3448/3448 [00:00<00:00, 28900.63 examples/s]Generating test split: 100% 308/308 [00:00<00:00, 8767.73 examples/s]Creating CSV from Arrow format: 100% 4/4 [00:00<00:00,  5.11ba/s]Creating CSV from Arrow format: 100% 1/1 [00:00<00:00, 15.27ba/s]Downloads complete
No path specified. Models will be saved in: "AutogluonModels/ag-20260202_094114"
Preset alias specified: 'high_v150' maps to 'high_quality_v150'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Pytorch Version:    2.9.0+cu126
CUDA Version:       12.6
GPU Memory:         GPU 0: 14.74/14.74 GB
Total GPU Memory:   Free: 14.74 GB, Allocated: 0.00 GB, Total: 14.74 GB
GPU Count:          1
Memory Avail:       9.19 GB / 12.67 GB (72.5%)
Disk Space Avail:   73.01 GB / 112.64 GB (64.8%)
===================================================
Presets specified: ['high_v150']
Using hyperparameters preset: hyperparameters='zeroshot_2025_12_18_cpu'
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
Beginning AutoGluon training ... Time limit = 6600s
AutoGluon will save models to "/content/AutogluonModels/ag-20260202_094114"
Train Data Rows:    3448
Train Data Columns: 3
Label Column:       clarity_label
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	3 unique label values:  ['Clear Reply', 'Ambivalent', 'Clear Non-Reply']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during Predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression', 'quantile'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    9414.87 MB
	Train Data (Original)  Memory Usage: 10.16 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['interview_question', 'interview_answer', 'question']
			CountVectorizer fit with vocabulary size = 8559
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('object', ['text']) : 3 | ['interview_question', 'interview_answer', 'question']
	Types of features in processed data (raw dtype, special dtypes):
		('category', ['text_as_category'])  :    3 | ['interview_question', 'interview_answer', 'question']
		('int', ['binned', 'text_special']) :   57 | ['interview_question.char_count', 'interview_question.word_count', 'interview_question.capital_ratio', 'interview_question.lower_ratio', 'interview_question.digit_ratio', ...]
		('int', ['text_ngram'])             : 8440 | ['__nlp__.000', '__nlp__.10', '__nlp__.10 years', '__nlp__.100', '__nlp__.100 percent', ...]
	22.8s = Fit runtime
	3 features in original data used to generate 8500 features in processed data.
	Train Data (Processed) Memory Usage: 55.71 MB (0.6% of available memory)
Data preprocessing and feature engineering runtime = 24.15s ...
AutoGluon will gauge predictive performance using evaluation metric: 'accuracy'
	To change this, specify the eval_metric parameter of Predictor()
Large model count detected (50 configs) ... Only displaying the first 3 models of each family. To see all, set `verbosity=3`.
User-specified model hyperparameters to be fit:
{
	'CAT': [{'ag_args': {'name_suffix': '_c1', 'priority': -1}}],
	'GBM_PREP': [{'ag.prep_params': [[[['ArithmeticFeatureGenerator', {}]], [['CategoricalInteractionFeatureGenerator', {'passthrough': True}], ['OOFTargetEncodingFeatureGenerator', {}]]]], 'ag.prep_params.passthrough_types': {'invalid_raw_types': ['category', 'object']}, 'ag_args': {'name_suffix': '_r13', 'priority': -2}, 'ag_args_ensemble': {'model_random_seed': 0, 'vary_seed_across_folds': True}, 'bagging_fraction': 0.9923026236907, 'bagging_freq': 1, 'cat_l2': 0.014290368488, 'cat_smooth': 1.8662939903973, 'extra_trees': True, 'feature_fraction': 0.5533919718605, 'lambda_l1': 0.914411672958, 'lambda_l2': 1.90439560009, 'learning_rate': 0.0193225778401, 'max_cat_to_onehot': 18, 'min_data_in_leaf': 28, 'min_data_per_group': 54, 'num_leaves': 64}, {'ag.prep_params': [[[['ArithmeticFeatureGenerator', {}]], [['CategoricalInteractionFeatureGenerator', {'passthrough': True}], ['OOFTargetEncodingFeatureGenerator', {}]]]], 'ag.prep_params.passthrough_types': {'invalid_raw_types': ['category', 'object']}, 'ag_args': {'name_suffix': '_r41', 'priority': -7}, 'ag_args_ensemble': {'model_random_seed': 0, 'vary_seed_across_folds': True}, 'bagging_fraction': 0.7215411996558, 'bagging_freq': 1, 'cat_l2': 1.887369154362, 'cat_smooth': 0.0278693980873, 'extra_trees': True, 'feature_fraction': 0.4247583287144, 'lambda_l1': 0.1129800247772, 'lambda_l2': 0.2623265718536, 'learning_rate': 0.0074201920651, 'max_cat_to_onehot': 9, 'min_data_in_leaf': 15, 'min_data_per_group': 10, 'num_leaves': 8}, {'ag.prep_params': [[[['ArithmeticFeatureGenerator', {}]], [['CategoricalInteractionFeatureGenerator', {'passthrough': True}], ['OOFTargetEncodingFeatureGenerator', {}]]]], 'ag.prep_params.passthrough_types': {'invalid_raw_types': ['category', 'object']}, 'ag_args': {'name_suffix': '_r31', 'priority': -10}, 'ag_args_ensemble': {'model_random_seed': 0, 'vary_seed_across_folds': True}, 'bagging_fraction': 0.9591526242875, 'bagging_freq': 1, 'cat_l2': 1.8962346412823, 'cat_smooth': 0.0215219089995, 'extra_trees': False, 'feature_fraction': 0.5791844062459, 'lambda_l1': 0.938461750637, 'lambda_l2': 0.9899852075056, 'learning_rate': 0.0397613094741, 'max_cat_to_onehot': 27, 'min_data_in_leaf': 1, 'min_data_per_group': 39, 'num_leaves': 16}],
	'GBM': [{'ag_args': {'name_suffix': '_r177', 'priority': -3}, 'bagging_fraction': 0.8769107816033, 'bagging_freq': 1, 'cat_l2': 0.3418014393813, 'cat_smooth': 15.4304556649114, 'extra_trees': True, 'feature_fraction': 0.4622189821941, 'lambda_l1': 0.2375070586896, 'lambda_l2': 0.3551561351804, 'learning_rate': 0.0178593900218, 'max_cat_to_onehot': 16, 'min_data_in_leaf': 3, 'min_data_per_group': 9, 'num_leaves': 39}, {'ag_args': {'name_suffix': '_r163', 'priority': -5}, 'bagging_fraction': 0.9783898288461, 'bagging_freq': 1, 'cat_l2': 0.1553395260142, 'cat_smooth': 0.0093122749318, 'extra_trees': False, 'feature_fraction': 0.5279825611461, 'lambda_l1': 0.0269274915833, 'lambda_l2': 0.8375250972309, 'learning_rate': 0.0113913650333, 'max_cat_to_onehot': 42, 'min_data_in_leaf': 3, 'min_data_per_group': 75, 'num_leaves': 84}, {'ag_args': {'name_suffix': '_r72', 'priority': -8}, 'bagging_fraction': 0.950146543918, 'bagging_freq': 1, 'cat_l2': 0.2159137242663, 'cat_smooth': 0.0638204395719, 'extra_trees': True, 'feature_fraction': 0.4044759649281, 'lambda_l1': 0.7661581500422, 'lambda_l2': 1.6041759693902, 'learning_rate': 0.0179845918984, 'max_cat_to_onehot': 11, 'min_data_in_leaf': 12, 'min_data_per_group': 3, 'num_leaves': 180}],
	'NN_TORCH': [{'activation': 'elu', 'ag_args': {'name_suffix': '_r37', 'priority': -4}, 'dropout_prob': 0.0889772897547275, 'hidden_size': 109, 'learning_rate': 0.02184363543226557, 'num_layers': 3, 'use_batchnorm': True, 'weight_decay': 3.1736637236578543e-10}, {'activation': 'elu', 'ag_args': {'name_suffix': '_r31', 'priority': -9}, 'dropout_prob': 0.013288954106470907, 'hidden_size': 81, 'learning_rate': 0.005340914647396153, 'num_layers': 4, 'use_batchnorm': False, 'weight_decay': 8.76216837077536e-05}, {'activation': 'elu', 'ag_args': {'name_suffix': '_r193', 'priority': -14}, 'dropout_prob': 0.2976404923811552, 'hidden_size': 131, 'learning_rate': 0.0038408014156739775, 'num_layers': 3, 'use_batchnorm': False, 'weight_decay': 0.01745189206113213}],
	'FASTAI': [{'ag_args': {'name_suffix': '_r25', 'priority': -6}, 'bs': 1024, 'emb_drop': 0.6167722379778131, 'epochs': 44, 'layers': [200, 100, 50], 'lr': 0.05344037785562929, 'ps': 0.48477211305443607}, {'ag_args': {'name_suffix': '_r162', 'priority': -11}, 'bs': 2048, 'emb_drop': 0.5474625640581479, 'epochs': 45, 'layers': [400, 200], 'lr': 0.0047438648957706655, 'ps': 0.07533239360470734}, {'ag_args': {'name_suffix': '_r147', 'priority': -20}, 'bs': 128, 'emb_drop': 0.6378380130337095, 'epochs': 48, 'layers': [200], 'lr': 0.058027179860229344, 'ps': 0.23253362133888375}],
}
User-specified callbacks (1): ['EarlyStoppingCountCallback']
EarlyStoppingCountCallback: Initializing patience to 18. Reason: num_rows_train=3448, patience_curve=[[100, 4], [500, 8], [2500, 15], [10000, 40], [100000, 100], None]
Fitting 50 L1 models, fit_strategy="sequential" ...
Fitting model: CatBoost_c1_BAG_L1 ... Training model for up to 6575.85s of the 6575.83s of remaining time.
	Warning: Potentially not enough memory to safely train model. Estimated to require 8.152 GB out of 9.001 GB available memory (90.571%)... (100.000% of avail memory is the max safe size)
	To avoid this warning, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.26 to avoid the warning)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 90.57% memory usage per fold, 90.57%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=90.57%)
/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py:2062: FutureWarning: Tip: In future versions of Ray, Ray will no longer override accelerator visible devices env var if num_gpus=0 or num_gpus=None (default). To enable this behavior and turn off this error message, set RAY_ACCEL_ENV_VAR_OVERRIDE_ON_ZERO=0
  warnings.warn(
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling `predictor.fit`.
	Not enough memory to train CatBoost_c1_BAG_L1... Skipping this model.
Fitting model: LightGBMPrep_r13_BAG_L1 ... Training model for up to 6549.59s of the 6549.57s of remaining time.
	Warning: Not enough memory to safely train model. Estimated to require 9.460 GB out of 8.054 GB available memory (117.463%)... (90.000% of avail memory is the max safe size)
	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.36 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Not enough memory to train LightGBMPrep_r13_BAG_L1... Skipping this model.
Fitting model: LightGBM_r177_BAG_L1 ... Training model for up to 6531.28s of the 6531.26s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 2 folds in parallel instead (Estimated 28.72% memory usage per fold, 57.43%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=28.72%)
	0.641	 = Validation score   (accuracy)
	403.84s	 = Training   runtime
	1.01s	 = Validation runtime
Fitting model: NeuralNetTorch_r37_BAG_L1 ... Training model for up to 6114.67s of the 6114.66s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=4.04%)
	0.6285	 = Validation score   (accuracy)
	106.3s	 = Training   runtime
	0.4s	 = Validation runtime
Fitting model: LightGBM_r163_BAG_L1 ... Training model for up to 5997.04s of the 5997.02s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 56.68% memory usage per fold, 56.68%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=56.68%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	0.6436	 = Validation score   (accuracy)
	863.52s	 = Training   runtime
	1.16s	 = Validation runtime
Fitting model: NeuralNetFastAI_r25_BAG_L1 ... Training model for up to 5121.09s of the 5121.08s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=7.42%)
	0.6224	 = Validation score   (accuracy)
	71.31s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: LightGBMPrep_r41_BAG_L1 ... Training model for up to 5038.20s of the 5038.18s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 2 folds in parallel instead (Estimated 32.14% memory usage per fold, 64.29%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=32.14%)
	Warning: Exception caused LightGBMPrep_r41_BAG_L1 to fail during training... Skipping this model.
		ray::_ray_fit() (pid=10439, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2201, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2085, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/stacker_ensemble_model.py", line 270, in _fit
    return super()._fit(X=X, y=y, time_limit=time_limit, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 393, in _fit
    self._fit_folds(
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 887, in _fit_folds
    fold_fitting_strategy.after_all_folds_scheduled()
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 796, in after_all_folds_scheduled
    self._run_parallel(X, y, X_pseudo, y_pseudo, model_base_ref, time_limit_fold, head_node_id)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 713, in _run_parallel
    self._process_fold_results(finished, unfinished, fold_ctx)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 666, in _process_fold_results
    raise processed_exception
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 619, in _process_fold_results
    out = self.ray.get(finished)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 2972, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 1031, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(LightGBMError): ray::_ray_fit() (pid=10439, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Fitting model: LightGBM_r72_BAG_L1 ... Training model for up to 5007.62s of the 5007.61s of remaining time.
	Warning: Not enough memory to safely train model. Estimated to require 9.339 GB out of 7.975 GB available memory (117.103%)... (90.000% of avail memory is the max safe size)
	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.35 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Not enough memory to train LightGBM_r72_BAG_L1... Skipping this model.
Fitting model: NeuralNetTorch_r31_BAG_L1 ... Training model for up to 4997.45s of the 4997.43s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=4.09%)
	0.6288	 = Validation score   (accuracy)
	93.07s	 = Training   runtime
	0.38s	 = Validation runtime
Fitting model: LightGBMPrep_r31_BAG_L1 ... Training model for up to 4893.51s of the 4893.49s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 44.95% memory usage per fold, 44.95%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=44.95%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	Warning: Exception caused LightGBMPrep_r31_BAG_L1 to fail during training... Skipping this model.
		ray::_ray_fit() (pid=11593, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2201, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2085, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/stacker_ensemble_model.py", line 270, in _fit
    return super()._fit(X=X, y=y, time_limit=time_limit, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 393, in _fit
    self._fit_folds(
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 887, in _fit_folds
    fold_fitting_strategy.after_all_folds_scheduled()
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 794, in after_all_folds_scheduled
    self._run_pseudo_sequential(X, y, X_pseudo, y_pseudo, model_base_ref, time_limit_fold, head_node_id)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 752, in _run_pseudo_sequential
    self._process_fold_results(finished[0], unfinished, fold_ctx)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 666, in _process_fold_results
    raise processed_exception
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 619, in _process_fold_results
    out = self.ray.get(finished)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 2972, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 1031, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(LightGBMError): ray::_ray_fit() (pid=11593, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Fitting model: NeuralNetFastAI_r162_BAG_L1 ... Training model for up to 4872.62s of the 4872.60s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=7.48%)
	0.6059	 = Validation score   (accuracy)
	73.95s	 = Training   runtime
	0.16s	 = Validation runtime
Fitting model: LightGBMPrep_r21_BAG_L1 ... Training model for up to 4787.43s of the 4787.41s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 41.62% memory usage per fold, 41.62%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=41.62%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	0.6473	 = Validation score   (accuracy)
	576.19s	 = Training   runtime
	1.07s	 = Validation runtime
Fitting model: LightGBM_r120_BAG_L1 ... Training model for up to 4197.75s of the 4197.74s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=7.80%)
	0.6363	 = Validation score   (accuracy)
	167.77s	 = Training   runtime
	1.03s	 = Validation runtime
Fitting model: NeuralNetTorch_r193_BAG_L1 ... Training model for up to 4016.18s of the 4016.16s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=4.12%)
	0.6079	 = Validation score   (accuracy)
	96.2s	 = Training   runtime
	0.47s	 = Validation runtime
Fitting model: NeuralNetTorch_r144_BAG_L1 ... Training model for up to 3908.47s of the 3908.45s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=4.11%)
	0.6267	 = Validation score   (accuracy)
	135.75s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: LightGBM_r6_BAG_L1 ... Training model for up to 3760.93s of the 3760.91s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=9.69%)
	0.6386	 = Validation score   (accuracy)
	217.71s	 = Training   runtime
	1.64s	 = Validation runtime
Fitting model: LightGBMPrep_r17_BAG_L1 ... Training model for up to 3529.08s of the 3529.06s of remaining time.
	Warning: Not enough memory to safely train model. Estimated to require 9.460 GB out of 7.950 GB available memory (118.995%)... (90.000% of avail memory is the max safe size)
	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=1.37 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Not enough memory to train LightGBMPrep_r17_BAG_L1... Skipping this model.
Fitting model: LightGBMPrep_r47_BAG_L1 ... Training model for up to 3518.20s of the 3518.18s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 69.68% memory usage per fold, 69.68%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=69.68%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	Warning: Exception caused LightGBMPrep_r47_BAG_L1 to fail during training... Skipping this model.
		ray::_ray_fit() (pid=20346, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2201, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2085, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/stacker_ensemble_model.py", line 270, in _fit
    return super()._fit(X=X, y=y, time_limit=time_limit, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 393, in _fit
    self._fit_folds(
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 887, in _fit_folds
    fold_fitting_strategy.after_all_folds_scheduled()
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 794, in after_all_folds_scheduled
    self._run_pseudo_sequential(X, y, X_pseudo, y_pseudo, model_base_ref, time_limit_fold, head_node_id)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 752, in _run_pseudo_sequential
    self._process_fold_results(finished[0], unfinished, fold_ctx)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 666, in _process_fold_results
    raise processed_exception
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 619, in _process_fold_results
    out = self.ray.get(finished)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 2972, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 1031, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(LightGBMError): ray::_ray_fit() (pid=20346, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Fitting model: LightGBMPrep_r1_BAG_L1 ... Training model for up to 3498.82s of the 3498.81s of remaining time.
	Warning: Not enough memory to safely train model. Estimated to require 14.099 GB out of 7.912 GB available memory (178.197%)... (90.000% of avail memory is the max safe size)
	To force training the model, specify the model hyperparameter "ag.max_memory_usage_ratio" to a larger value (currently 1.0, set to >=2.03 to avoid the error)
		To set the same value for all models, do the following when calling predictor.fit: `predictor.fit(..., ag_args_fit={"ag.max_memory_usage_ratio": VALUE})`
		Setting "ag.max_memory_usage_ratio" to values above 1 may result in out-of-memory errors. You may consider using a machine with more memory as a safer alternative.
	Not enough memory to train LightGBMPrep_r1_BAG_L1... Skipping this model.
Fitting model: NeuralNetFastAI_r147_BAG_L1 ... Training model for up to 3487.92s of the 3487.90s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=7.55%)
	0.6206	 = Validation score   (accuracy)
	103.67s	 = Training   runtime
	0.2s	 = Validation runtime
Fitting model: LightGBM_r184_BAG_L1 ... Training model for up to 3372.81s of the 3372.79s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=5.91%)
	0.6099	 = Validation score   (accuracy)
	374.85s	 = Training   runtime
	0.91s	 = Validation runtime
Fitting model: NeuralNetTorch_r82_BAG_L1 ... Training model for up to 2984.39s of the 2984.38s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=4.11%)
	0.6273	 = Validation score   (accuracy)
	127.92s	 = Training   runtime
	0.53s	 = Validation runtime
Fitting model: LightGBM_r46_BAG_L1 ... Training model for up to 2844.92s of the 2844.90s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 13.45% memory usage per fold, 53.78%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=13.45%)
	0.645	 = Validation score   (accuracy)
	398.08s	 = Training   runtime
	1.23s	 = Validation runtime
Fitting model: LightGBM_r68_BAG_L1 ... Training model for up to 2433.39s of the 2433.37s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 1 folds in parallel instead (Estimated 74.60% memory usage per fold, 74.60%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1 workers, per: cpus=1, gpus=0, memory=74.60%)
		Switching to pseudo sequential ParallelFoldFittingStrategy to avoid Python memory leakage.
		Overrule this behavior by setting fold_fitting_strategy to 'sequential_local' in ag_args_ensemble when when calling `predictor.fit`
	0.6441	 = Validation score   (accuracy)
	842.09s	 = Training   runtime
	1.13s	 = Validation runtime
Fitting model: NeuralNetFastAI_r192_BAG_L1 ... Training model for up to 1579.25s of the 1579.23s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=7.56%)
	0.5769	 = Validation score   (accuracy)
	73.97s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: LightGBMPrep_r19_BAG_L1 ... Training model for up to 1493.50s of the 1493.48s of remaining time.
	Memory not enough to fit 8 folds in parallel. Will train 2 folds in parallel instead (Estimated 36.80% memory usage per fold, 73.61%/80.00% total).
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=36.80%)
	Warning: Exception caused LightGBMPrep_r19_BAG_L1 to fail during training... Skipping this model.
		ray::_ray_fit() (pid=32220, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Detailed Traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2201, in _train_and_save
    model = self._train_single(**model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/trainer/abstract_trainer.py", line 2085, in _train_single
    model = model.fit(X=X, y=y, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test, total_resources=total_resources, **model_fit_kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/stacker_ensemble_model.py", line 270, in _fit
    return super()._fit(X=X, y=y, time_limit=time_limit, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 393, in _fit
    self._fit_folds(
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/bagged_ensemble_model.py", line 887, in _fit_folds
    fold_fitting_strategy.after_all_folds_scheduled()
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 796, in after_all_folds_scheduled
    self._run_parallel(X, y, X_pseudo, y_pseudo, model_base_ref, time_limit_fold, head_node_id)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 713, in _run_parallel
    self._process_fold_results(finished, unfinished, fold_ctx)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 666, in _process_fold_results
    raise processed_exception
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 619, in _process_fold_results
    out = self.ray.get(finished)
          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/auto_init_hook.py", line 22, in auto_init_wrapper
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/client_mode_hook.py", line 104, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 2972, in get
    values, debugger_breakpoint = worker.get_objects(
                                  ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ray/_private/worker.py", line 1031, in get_objects
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(LightGBMError): ray::_ray_fit() (pid=32220, ip=172.28.0.12)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/ensemble/fold_fitting_strategy.py", line 473, in _ray_fit
    fold_model.fit(X=X_fold, y=y_fold, X_val=X_val_fold, y_val=y_val_fold, time_limit=time_limit_fold, **resources, **kwargs_fold)
  File "/usr/local/lib/python3.12/dist-packages/autogluon/core/models/abstract/abstract_model.py", line 1125, in fit
    out = self._fit(**kwargs)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_model.py", line 335, in _fit
    self.model = train_lgb_model(early_stopping_callback_kwargs=early_stopping_callback_kwargs, **train_params)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/autogluon/tabular/models/lgb/lgb_utils.py", line 134, in train_lgb_model
    return lgb.train(**train_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/engine.py", line 297, in train
    booster = Booster(params=params, train_set=train_set)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 3660, in __init__
    _safe_call(
  File "/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py", line 313, in _safe_call
    raise LightGBMError(_LIB.LGBM_GetLastError().decode("utf-8"))
lightgbm.basic.LightGBMError: Forced splits file includes feature index 0, but maximum feature index in dataset is -1
Fitting model: NeuralNetTorch_r39_BAG_L1 ... Training model for up to 1464.63s of the 1464.61s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, memory=4.14%)
	0.6276	 = Validation score   (accuracy)
	101.21s	 = Training   runtime
	0.42s	 = Validation runtime
EarlyStoppingCountCallback: Early stopping trainer fit for level=1. Reason: Fit 18 models (max_models=18)
Fitting model: WeightedEnsemble_L2 ... Training model for up to 657.58s of the 1351.92s of remaining time.
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=0.0/7.5 GB
	Ensemble Weights: {'LightGBMPrep_r21_BAG_L1': 1.0}
	0.6473	 = Validation score   (accuracy)
	0.14s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 5248.39s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 404.3 rows/s (431 batch size)
Automatically performing refit_full as a post-fit operation (due to `.fit(..., refit_full=True)`
Refitting models via `predictor.refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix "_FULL" and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `predictor.fit` call.
	To learn more, refer to the `.refit_full` method docstring which explains how "_FULL" models differ from normal models.
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r177_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=2.2/7.9 GB
	25.66s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r37_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.3/7.8 GB
	5.95s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r163_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=4.4/7.7 GB
	42.93s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI_r25_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.5/7.7 GB
No improvement since epoch 0: early stopping
	3.71s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r31_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.3/7.7 GB
	4.84s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI_r162_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.5/7.7 GB
No improvement since epoch 2: early stopping
	3.0s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBMPrep_r21_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=3.2/7.7 GB
	36.83s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r120_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=0.6/7.6 GB
	16.44s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r193_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.3/7.7 GB
	7.67s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r144_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.3/7.7 GB
	10.33s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r6_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=0.7/7.7 GB
	18.9s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI_r147_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.5/7.7 GB
No improvement since epoch 0: early stopping
	6.94s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r184_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=0.4/7.6 GB
	33.34s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r82_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.3/7.7 GB
	9.53s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r46_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=1.0/7.7 GB
	35.32s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM_r68_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=2, gpus=0, mem=5.7/7.6 GB
	53.78s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetFastAI_r192_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.5/7.7 GB
No improvement since epoch 1: early stopping
	3.98s	 = Training   runtime
Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorch_r39_BAG_L1_FULL ...
	Fitting 1 model on all data | Fitting with cpus=1, gpus=0, mem=0.3/7.6 GB
	8.1s	 = Training   runtime
Fitting model: WeightedEnsemble_L2_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'LightGBMPrep_r21_BAG_L1': 1.0}
	0.14s	 = Training   runtime
Updated best model to "LightGBMPrep_r21_BAG_L1_FULL" (Previously "WeightedEnsemble_L2"). AutoGluon will default to using "LightGBMPrep_r21_BAG_L1_FULL" for predict() and predict_proba().
Refit complete, total runtime = 422.42s ... Best model: "LightGBMPrep_r21_BAG_L1_FULL"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/AutogluonModels/ag-20260202_094114")
Loaded data from: evaluation.csv | Columns = 20 / 20 | Rows = 237 -> 237

```